# Treebank Alignments
1. Align Sentences: SPMRL <--> UD
1. Align original sentence tokens to morphemes in SPMRL & UD
1. Align Morphemes: SPMRL <--> UD
1. Create tests to run when commiting new versions of TBs 
1. Upload official train-dev-test splits for SPMRL & UD
   1. Add sentence alignment table between SPMRL & UD (also add standard sentence IDs to the sentence number comment)
   1. Add morpheme alignment table between SPMRL & UD
   1. Add token-to-morpheme alignment table
   1. Add NER tag tables for SPMRL & UD

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
%matplotlib inline

In [6]:
import pandas as pd
import numpy as np

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')

In [8]:
def make_conll_df(path, add_head_stuff=False):
    # CoNLL file is tab delimeted with no quoting
    # quoting=3 is csv.QUOTE_NONE
    df = (pd.read_csv(path, sep='\t', header=None, quoting=3, comment='#',
                names = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC'])
                # add sentence labels
                .assign(sent = lambda x: (x.ID==1).cumsum())
                # replace bad root dependency tags
                .replace({'DEPREL': {'prd': 'ROOT'}})
               )
    
    if add_head_stuff:
        df = df.merge(df[['ID', 'FORM', 'sent', 'UPOS']].rename(index=str, columns={'FORM': 'head_form', 'UPOS': 'head_upos'}).set_index(['sent', 'ID']),
               left_on=['sent', 'HEAD'], right_index=True, how='left')
    return df


## 1. Align Sentences: SPMRL <--> UD


In [6]:
ner = (pd.read_csv('curation.csv.gz')
      .assign(sent=lambda x: x.file+'_'+x.sent.astype(str)))

In [7]:
def get_orig_sent(s):
    start = int(s.split('-')[0].split('_')[1])
    sent = int(s.split('.')[1].split('_')[1])
    orig_sent = start+sent-1
    return orig_sent

In [8]:
ner['orig_sent'] = ner.sent.apply(get_orig_sent)

In [9]:
ner.head()

,sent_tok_num,tok_offset,token,FEAT_gender,FEAT_number,FEAT_case,FEAT_degree,FEAT_transitivity,FEAT_tense,FEAT_mood,...,dep_lex_morph_pos,dep_arc,EXTRA,sent,ner_layers,ner_type,is_ner,biose,file,orig_sent
0,1-1,0-5,עשרות,Fem,Plur,*,*,*,*,*,...,1-2,NaN,NaN,dev_1-100.tsv_1,0,_,False,O,dev_1-100.tsv,1
1,1-2,6-11,אנשים,Masc,Plur,*,*,*,*,*,...,1-3,NaN,NaN,dev_1-100.tsv_1,0,_,False,O,dev_1-100.tsv,1
2,1-3,12-18,מגיעים,Masc,Plur,*,*,*,*,*,...,1-3,NaN,NaN,dev_1-100.tsv_1,0,_,False,O,dev_1-100.tsv,1
3,1-4,19-20,מ,_,_,_,_,_,_,_,...,1-5,NaN,NaN,dev_1-100.tsv_1,0,_,False,O,dev_1-100.tsv,1
4,1-5,21-27,תאילנד,_,_,_,_,_,_,_,...,1-3,NaN,NaN,dev_1-100.tsv_1,1,GPE,True,S-GPE,dev_1-100.tsv,1


In [10]:
ner_sents = ner.groupby('orig_sent').apply(lambda x: ' '.join(x.token.to_list()))
ner_sents.head()

orig_sent
1    עשרות אנשים מגיעים מ תאילנד ל ישראל כש הם נרשמ...
2    תופעה זו התבררה אתמול ב וועדת ה עבודה ו ה רווח...
3    יו"ר ה וועדה , ח"כ אורה נמיר ( מערך ) , טענה כ...
4    מ צד אחד רוצה ה אוצר להוריד את שכר ה מינימום ,...
5    נמיר הודיעה כי תפנה ל שרי ה פנים ו ה עבודה ו ה...
dtype: object

In [11]:
#TODO:
# ud_sents = RELEASE FROM GITHUB
# spmrl_sents = from AMIT / other version?

### UD sentences

In [30]:
from conllu import parse

In [31]:
with open('../UD_Hebrew-HTB/he_htb-ud-dev.conllu', 'r', encoding='utf8') as f:
    ud_dev = parse(f.read())
with open('../UD_Hebrew-HTB/he_htb-ud-test.conllu', 'r', encoding='utf8') as f:
    ud_test = parse(f.read())
with open('../UD_Hebrew-HTB/he_htb-ud-train.conllu', 'r', encoding='utf8') as f:
    ud_train = parse(f.read())

In [25]:
ud_dev[0].metadata

OrderedDict([('sent_id', '1'),
             ('text',
              'עשרות אנשים מגיעים מתאילנד לישראל כשהם נרשמים כמתנדבים, אך למעשה משמשים עובדים שכירים זולים.')])

In [989]:
list(ud_dev[2])[:3]

[OrderedDict([('id', 1),
              ('form', 'יו"ר'),
              ('lemma', 'יו"ר'),
              ('upostag', 'NOUN'),
              ('xpostag', 'NOUN'),
              ('feats',
               OrderedDict([('Abbr', 'Yes'),
                            ('Definite', 'Cons'),
                            ('Gender', 'Masc'),
                            ('Number', 'Sing')])),
              ('head', 12),
              ('deprel', 'nsubj'),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', (2, '-', 3)),
              ('form', 'הוועדה'),
              ('lemma', '_'),
              ('upostag', '_'),
              ('xpostag', None),
              ('feats', None),
              ('head', None),
              ('deprel', '_'),
              ('deps', None),
              ('misc', OrderedDict([('SpaceAfter', 'No')]))]),
 OrderedDict([('id', 2),
              ('form', 'ה'),
              ('lemma', 'ה'),
              ('upostag', 'DET'),
              ('xpostag', 'DET

In [104]:
def get_conllu_form_sent(tl):
    return tl.metadata['sent_id'], ' '.join([t['form'] for t in tl if type(t['id']) is int])

ud_sents = [get_conllu_form_sent(tl) for tl in ud_dev] + [get_conllu_form_sent(tl) for tl in ud_train] + [get_conllu_form_sent(tl) for tl in ud_test]
ud_sents = list(zip(*ud_sents))
ud_sents = pd.Series(index=ud_sents[0], data=ud_sents[1])
ud_sents.head()

1    עשרות אנשים מגיעים מ תאילנד ל ישראל כש הם נרשמ...
2    תופעה זו התבררה אתמול ב וועדת ה עבודה ו ה רווח...
3    יו"ר ה וועדה , ח"כ אורה נמיר ( מערך ) , טענה כ...
4    מ צד אחד רוצה ה אוצר להוריד את שכר ה מינימום ,...
5    נמיר הודיעה כי תפנה ל שרי ה פנים ו ה עבודה ו ה...
dtype: object

### SPMRL sentences

In [655]:
def get_conllu_form_sent_clean(tl):
    skip_up_to = -1
    sent_id = tl.metadata['sent_id']
    toks = []
    for i, t in enumerate(tl):
        tok_id = t['id']
        if type(tok_id) is tuple:
            first_tok_id=tok_id[0]
            last_tok_id = tok_id[2]
            tok_len = last_tok_id - first_tok_id
        else:
            first_tok_id=tok_id
            
        if first_tok_id>skip_up_to:
            if type(tok_id) is tuple and (tl[i+tok_len+1]['form'].startswith('_') or tl[i+tok_len+1]['form'].endswith('_')):
                toks.append(t['form'].strip('_'))                                           
                skip_up_to = last_tok_id
                #print(skip_up_to)
            elif type(tok_id) is int:
                toks.append(t['form'].strip('_'))
            
    return sent_id, ' '.join(toks)

ud_sents_clean = [get_conllu_form_sent_clean(tl) for tl in ud_dev] + [get_conllu_form_sent_clean(tl) for tl in ud_train] + [get_conllu_form_sent_clean(tl) for tl in ud_test]
ud_sents_clean = list(zip(*ud_sents_clean))
ud_sents_clean = pd.Series(index=ud_sents_clean[0], data=ud_sents_clean[1])
ud_sents_clean.head()

1    עשרות אנשים מגיעים מ תאילנד ל ישראל כש הם נרשמ...
2    תופעה זו התבררה אתמול ב וועדת ה עבודה ו ה רווח...
3    יו"ר ה וועדה , ח"כ אורה נמיר ( מערך ) , טענה כ...
4    מ צד אחד רוצה ה אוצר להוריד את שכר ה מינימום ,...
5    נמיר הודיעה כי תפנה ל שרי ה פנים ו ה עבודה ו ה...
dtype: object

In [45]:
sp_test = make_conll_df('hebtb_spmrl/test_hebtb-gold.conll')
sp_dev = make_conll_df('hebtb_spmrl/dev_hebtb-gold.conll')
sp_train = make_conll_df('hebtb_spmrl/train_hebtb-gold.conll')
sp_dev.sent.nunique(), sp_train.sent.nunique(), sp_test.sent.nunique()

(500, 4937, 716)

In [46]:
sp_dev['orig_sent'] = sp_dev.sent
sp_dev['s'] = sp_dev.sent

sp_train['orig_sent'] = sp_train.sent + 500
sp_test['orig_sent'] = sp_test.sent + 500 + 4937

sp_dev.orig_sent.min(), sp_dev.orig_sent.max(), sp_train.orig_sent.min(), sp_train.orig_sent.max(), sp_test.orig_sent.min(), sp_test.orig_sent.max()



(1, 500, 501, 5437, 5438, 6153)

In [1529]:
sp_sents = pd.concat([sp_dev, sp_train, sp_test])
sp_sents = sp_sents.groupby('orig_sent').apply(lambda x: ' '.join(x.FORM.to_list()))
sp_sents.head()

orig_sent
1    עשרות אנשים מגיעים מ תאילנד ל ישראל כש הם נרשמ...
2    תופעה זו התבררה אתמול ב וועדת ה עבודה ו ה רווח...
3    יו"ר ה וועדה , ח"ך אורה נמיר ( מערך ) , טענה כ...
4    מ צד אחד רוצה ה אוצר להוריד את שכר ה מינימום ,...
5    נמיר הודיעה כי תפנה ל שרי ה פנים ו ה עבודה ו ה...
dtype: object

### ud - ner alignment

In [ ]:
from fuzzywuzzy import fuzz


In [520]:
for u, n in zip(ud_sents, ner_sents):
    if fuzz.ratio(u, n)<80:
        print (u)

Fully aligned

### UD - SPMRL alignment

In [477]:
ud_sents_clean.to_csv('ud_sents_clean.csv')
sp_sents.to_csv('sp_sents.csv')

/home/dan/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/home/dan/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [513]:
# ud_sents_clean: '4828', '4829' - duplicates
# ud_sents_clean: '5432'-'5518' - many duplicates with '4126'+

In [514]:
remove = ['1616', '2405', '2837', '3301', '3314', '3379', '3549', '3749', '3785', '4058', '4069', '4210', '4240', '4829', '5071', '5099', '5164', '5169', '5282']
duplicate = ['5432', '5433', '5435', '5436', '5439', '5444', '5445', '5447', '5449', '5450', '5451', '5452', '5454', '5455', '5461', '5462', '5468', '5469', '5470', '5473', '5474', '5475', '5476', '5477', '5478', '5479', '5480', '5481', '5482', '5483', '5484', '5485', '5486', '5488', '5489', '5490', '5491', '5492', '5493', '5494', '5496', '5497', '5498', '5499']
remove += duplicate

In [526]:
streak_type = 'good'
streak_start=0
for i, (u, s) in enumerate(zip(ud_sents_clean.drop(remove), sp_sents)):
    if fuzz.ratio(u, s)<70:
        res = 'bad'
        print(u)
        print(s)
    else: res='good'
    if streak_type!=res:
        if streak_start==i-1:
            streak_desc = f'{streak_type}: {i-1}'
        else:
            streak_desc = f'{streak_type}: {streak_start} - {i-1}'
        streak_start = i
        streak_type = res
        print (streak_desc)

streak_desc = f'{streak_type}: {streak_start} - {i-1}'
print (streak_desc)

לאחר סדרה לא קצרה של ישיבות , שבהן פרשה ה תביעה את ראיותיה , הגיעו שני ה צדדים ל הסכם : גיליון ה אישום תוקן הושמטה ממנו ה פסקה ש הזכירה את מותו של מובארק ו נקבע כי ה יריות ש ירו ה שניים לא גרמו ל סיכון חיי אדם .
לאחר סדרה לא קצרה של ישיבות , ש ב הן פרשה ה תביעה את ראיותיה , הגיעו שני ה צדדים ל הסכם : גיליון ה אישום תוקן הושמטה ממן הוא ה פסקה ש הזכירה את מותו של מובארק ו נקבע כי ה יריות ש ירו ה שניים לא גרמו ל סיכון חיי אדם .
good: 0 - 1373
bad: 1374
ח"כ ראובן ריבלין , ש התמודד על ה תפקיד , ביקש לפחות להתמנות כ חבר ב וועדת חוץ ו ביטחון , בעקבות פינוי מקום ל ה סיעה ב וועדה זו בשל פרישתו של ה ח"כ לשעבר זלמן שובל , אך ה סיעה החליטה לדחות את ה החלטה ב ה עניין .
ח"ך ראובן ריבלין , ש התמודד על ה תפקיד , ביקש לפחות להתמנות כ חבר ב וועדת חוץ ו ביטחון , בעקבות פינוי מקום ל ה סיעה ב וועדה זו בשל פרישתו של ה ח"ך לשעבר זלמן שובל , אך ה סיעה החליטה לדחות את ה החלטה ב ה עניין .
good: 1375 - 1868
bad: 1869
ועדת ה עבודה ו ה רווחה של ה כנסת ב ראשות ח"כ אורה נמיר אישרה תיקון ל חוק מד"א , ה אמור לאפשר ל ה

In [517]:
ud_sents_clean.drop(remove).iloc[5315], sp_sents.iloc[5315]

('ב התייחסו ל התקפה אמריקאית אפשרית אמר משאט : " אתם מטרידים את ה בחורים שלכם על לא דבר ו ה התקפה תביא את ה עוינות של כל ה ערבים ו כל ה מוסלמים , מפני ש אתם הולכים לבצע פעולת תוקפנות נגד עיראק מ מדינה מוסלמית אחרת , מ ארץ קדושה " .',
 'ב התייחסו ל התקפה אמריקאית אפשרית אמר משאט : " אתם מטרידים את ה בחורים של אתמ על לא דבר ו ה התקפה תביא את ה עוינות של כל ה ערבים ו כל ה מוסלמים , מפני ש אתם הולכים לבצע פעולת תוקפנות נגד עיראק מ מדינה מוסלמית אחרת , מ ארץ קדושה " .')

In [522]:
i = streak_start
ud_sents_clean.iloc[i+len(remove)-3:i+len(remove)+3]

5377    זאת בשל ה אי - ודאות לגבי יכולתה של ה ממשלה לג...
5378    כמו כן קיימת אי - בהירות לגבי סכום ה הוצאות ה ...
5379                 תקציב 1991 יגיע ל כ 70 מיליארד שקל .
5380    סעיף תקציב ה ביטחון עדיין לא סוכם , כי משרד ה ...
5381    גם ב ה לובי ה חברתי ב ה כנסת הובעה באחרונה התנ...
5382    ה דולר נחלש ב יחס ל מרבית ה מטבעות ה עיקריים ב...
dtype: object

In [523]:
sp_sents.iloc[i-3:i+3]

orig_sent
5314    ה שגריר ה עיראקי ב וואשינגטון , מוחמד צאדק אל ...
5315    ה שגריר אמר את דבריו כמה שעות לאחר הודעת ה נשי...
5316    ב התייחסו ל התקפה אמריקאית אפשרית אמר משאט : "...
5317    ב מערכת ה ביטחון הוחל ב יישום ה הגבלות על תושב...
5318    מערכת ה ביטחון העבירה ל ה משטרה רשימה של כ 2,0...
5319                        הם יקבלו תעודות זהות ירוקות .
dtype: object

In [661]:
align_map = (pd.DataFrame(list(zip(ud_sents_clean.drop(remove).index, sp_sents.index)), columns=['ud', 'spmrl'])
             .assign(shft=lambda x: x.ud.astype(int)-x.spmrl)
             .assign(shift_change = lambda x: x.shft.ne(x.shft.shift(fill_value=0) ))
             .assign(global_sent_id = lambda x: x.index+1)
            )
align_map[align_map.shift_change]

,ud,spmrl,shft,shift_change,global_sent_id
1615,1617,1616,1,True,1616
2403,2406,2404,2,True,2404
2834,2838,2835,3,True,2835
3297,3302,3298,4,True,3298
3309,3315,3310,5,True,3310
3373,3380,3374,6,True,3374
3542,3550,3543,7,True,3543
3741,3750,3742,8,True,3742
3776,3786,3777,9,True,3777
4048,4059,4049,10,True,4049


In [662]:
align_map[align_map.global_sent_id!=align_map.spmrl]

,ud,spmrl,shft,shift_change,global_sent_id


### mark duplicate sentence pairs

#### UD

In [567]:
all_sim = {}
reverse_tested = {}
for i, s in ud_sents_clean.iteritems():
    similar = []
    for j, t in ud_sents_clean.drop([str(x+1) for x in range(int(i)-1)]) .iteritems():
        if len(s.split())>5 and len(t.split())>5 and abs(len(t.split())-len(s.split()))<10 and fuzz.ratio(s, t)>=70:
            similar.append(j)
    if len(similar)>1:
        print(i, similar)
    all_sim[i] = similar
    
all_sim

296 ['296', '326']
530 ['530', '554']
531 ['531', '555']
617 ['617', '861']
813 ['813', '819']
1580 ['1580', '5531']
1712 ['1712', '5817']
1913 ['1913', '2315']
1914 ['1914', '2316']
1915 ['1915', '2317']
1916 ['1916', '2318']
2106 ['2106', '2288', '2325', '2405', '3379', '3549', '3785', '4941', '5002', '5020', '5169', '5282']
2234 ['2234', '2235']
2288 ['2288', '2325', '2405', '3379', '3549', '3785', '4941', '5002', '5020', '5169', '5282']
2325 ['2325', '2405', '3379', '3549', '3785', '4941', '5002', '5020', '5169', '5282']
2405 ['2405', '3379', '3549', '3785', '4941', '5002', '5020', '5169', '5282']
2429 ['2429', '5379']
2526 ['2526', '2555', '2749', '2753']
2528 ['2528', '2555', '2749']
2529 ['2529', '2790', '2807']
2555 ['2555', '2557', '2749']
2749 ['2749', '2753']
2754 ['2754', '2786', '2804']
2799 ['2799', '2815']
3078 ['3078', '3082']
3127 ['3127', '3128']
3379 ['3379', '3549', '3785', '4941', '5002', '5020', '5169', '5282']
3549 ['3549', '3785', '4941', '5002', '5020', '5169',

{'1': ['1'],
 '2': ['2'],
 '3': ['3'],
 '4': ['4'],
 '5': ['5'],
 '6': ['6'],
 '7': ['7'],
 '8': ['8'],
 '9': ['9'],
 '10': ['10'],
 '11': ['11'],
 '12': ['12'],
 '13': ['13'],
 '14': ['14'],
 '15': ['15'],
 '16': ['16'],
 '17': ['17'],
 '18': ['18'],
 '19': ['19'],
 '20': ['20'],
 '21': ['21'],
 '22': ['22'],
 '23': ['23'],
 '24': ['24'],
 '25': ['25'],
 '26': ['26'],
 '27': ['27'],
 '28': ['28'],
 '29': ['29'],
 '30': ['30'],
 '31': ['31'],
 '32': ['32'],
 '33': ['33'],
 '34': ['34'],
 '35': ['35'],
 '36': ['36'],
 '37': ['37'],
 '38': ['38'],
 '39': ['39'],
 '40': ['40'],
 '41': ['41'],
 '42': ['42'],
 '43': [],
 '44': ['44'],
 '45': ['45'],
 '46': [],
 '47': ['47'],
 '48': ['48'],
 '49': ['49'],
 '50': ['50'],
 '51': ['51'],
 '52': ['52'],
 '53': ['53'],
 '54': ['54'],
 '55': ['55'],
 '56': ['56'],
 '57': ['57'],
 '58': ['58'],
 '59': ['59'],
 '60': ['60'],
 '61': [],
 '62': ['62'],
 '63': ['63'],
 '64': ['64'],
 '65': ['65'],
 '66': ['66'],
 '67': ['67'],
 '68': ['68'],
 '69': ['6

In [569]:
all_sim_ud = all_sim.copy()

In [575]:
len([s  for s in all_sim_ud.values() if len(s)>1])

114

In [653]:
[s  for s in all_sim_ud.values() if len(s)>1]

[['296', '326'],
 ['530', '554'],
 ['531', '555'],
 ['617', '861'],
 ['813', '819'],
 ['1580', '5531'],
 ['1712', '5817'],
 ['1913', '2315'],
 ['1914', '2316'],
 ['1915', '2317'],
 ['1916', '2318'],
 ['2106',
  '2288',
  '2325',
  '2405',
  '3379',
  '3549',
  '3785',
  '4941',
  '5002',
  '5020',
  '5169',
  '5282'],
 ['2234', '2235'],
 ['2288',
  '2325',
  '2405',
  '3379',
  '3549',
  '3785',
  '4941',
  '5002',
  '5020',
  '5169',
  '5282'],
 ['2325',
  '2405',
  '3379',
  '3549',
  '3785',
  '4941',
  '5002',
  '5020',
  '5169',
  '5282'],
 ['2405', '3379', '3549', '3785', '4941', '5002', '5020', '5169', '5282'],
 ['2429', '5379'],
 ['2526', '2555', '2749', '2753'],
 ['2528', '2555', '2749'],
 ['2529', '2790', '2807'],
 ['2555', '2557', '2749'],
 ['2749', '2753'],
 ['2754', '2786', '2804'],
 ['2799', '2815'],
 ['3078', '3082'],
 ['3127', '3128'],
 ['3379', '3549', '3785', '4941', '5002', '5020', '5169', '5282'],
 ['3549', '3785', '4941', '5002', '5020', '5169', '5282'],
 ['3616', 

In [1536]:
all_sim_ud_full = {}
for k, v in all_sim_ud.items():
    for i in v+[k]:
        if i not in all_sim_ud_full:
            dups = set((v+[k]))
            #dups.remove(i)
            all_sim_ud_full[i] = dups
all_sim_ud_full = {k:v for k,v in all_sim_ud_full.items() if len(v)>1}


In [986]:
ud_sents_clean['530']

'אתמול הושלכו ששה בקבוקי תבערה על סיורים של צה"ל ב רפיח .'

In [987]:
ud_sents_clean['554']

'אתמול הושלכו ארבעה בקבוקי תבערה על כוחות של צה"ל ב רפיח .'

In [1549]:
dup_id = ud_sents_clean[ud_sents_clean.duplicated(keep=False)].index
done = []
for k, v in all_sim_ud_full.items():
    if k not in done and k not in dup_id:
        print (ud_sents_clean[v])
        done.extend(v)

326    בראדלי ניצח לבסוף ב הפרש של שלושה אחוזים .
296         הוא נוצח השבוע ב הפרש של שני אחוזים .
dtype: object
530    אתמול הושלכו ששה בקבוקי תבערה על סיורים של צה"...
554    אתמול הושלכו ארבעה בקבוקי תבערה על כוחות של צה...
dtype: object
555    ה בקבוקים התנפצו , אך לא גרמו ל כל נזק .
531       ה בקבוקים התנפצו , אך לא היו נפגעים .
dtype: object
861    איני יכולה שלא להתריס בפניו על משפט אחד ב קטע ...
617    ו זה מה ש מביא את שגב ל ה פסקה ה לא - תיאמן ה ...
dtype: object
819    שאלנו על ה פולנים : האם יכלו לעשות יותר מ ש עש...
813    שאלנו אותו על ה יהודים : האם יכלו לעשות יותר מ...
dtype: object
5531      פיניקס ( אי - פי ) .
1580    ניו יורק ( אי - פי ) .
dtype: object
5817    ה דראמה ה גדולה החלה 14 שניות ל ה סיום .
1712          ה דרמה הגיעה לשיאה ב דקות ה סיום .
dtype: object
2315    ה קהילה ה יהודית של יוצאי קובה ב ארה"ב מארגנת ...
1913    ה קהילה ה יהודית ה קובנית ב ארה"ב מארגנת מסע ה...
dtype: object
1914    מ דיווח ש הגיע אתמול ל משרד ה תיירות ב ירושלים...
2316    מ דיווח 

#### SPMRL

In [570]:
sp_sents.head()

orig_sent
1    עשרות אנשים מגיעים מ תאילנד ל ישראל כש הם נרשמ...
2    תופעה זו התבררה אתמול ב וועדת ה עבודה ו ה רווח...
3    יו"ר ה וועדה , ח"ך אורה נמיר ( מערך ) , טענה כ...
4    מ צד אחד רוצה ה אוצר להוריד את שכר ה מינימום ,...
5    נמיר הודיעה כי תפנה ל שרי ה פנים ו ה עבודה ו ה...
dtype: object

In [ ]:
all_sim_spmrl = {}
reverse_tested = {}
for i, s in sp_sents.iteritems():
    similar = []
    for j, t in sp_sents.drop([x+1 for x in range(int(i)-1)]) .iteritems():
        if len(s.split())>5 and len(t.split())>5 and abs(len(t.split())-len(s.split()))<10 and fuzz.ratio(s, t)>=70:
            similar.append(j)
    if len(similar)>1:
        print(i, similar)
    all_sim_spmrl[i] = similar
    
all_sim_spmrl

In [ ]:
len([s  for s in all_sim_spmrl.values() if len(s)>1])

In [1541]:
with open('align/all_sim_spmrl.pkl', 'rb') as f:
    all_sim_spmrl = pickle.load(f) 

In [1542]:
all_sim_sp_full = {}
for k, v in all_sim_spmrl.items():
    for i in v+[k]:
        if i not in all_sim_sp_full:
            dups = set((v+[k]))
            #dups.remove(i)
            all_sim_sp_full[i] = dups
all_sim_sp_full = {k:v for k,v in all_sim_sp_full.items() if len(v)>1}

In [1551]:
dup_id_sp = sp_sents[sp_sents.duplicated(keep=False)].index
done_sp = []
for k, v in all_sim_sp_full.items():
    if k not in done_sp and k not in dup_id_sp:
        print (sp_sents.loc[v])
        done_sp.extend(v)

orig_sent
296         הוא נוצח השבוע ב הפרש של שני אחוזים .
326    בראדלי ניצח לבסוף ב הפרש של שלושה אחוזים .
dtype: object
orig_sent
530    אתמול הושלכו ששה בקבוקי תבערה על סיורים של צה"...
554    אתמול הושלכו ארבעה בקבוקי תבערה על כוחות של צה...
dtype: object
orig_sent
555    ה בקבוקים התנפצו , אך לא גרמו ל כל נזק .
531       ה בקבוקים התנפצו , אך לא היו נפגעים .
dtype: object
orig_sent
617    ו זה מה ש מביא את שגב ל ה פסקה ה לא - תיאמן ה ...
861    איני יכולה שלא להתריס ב פניו על משפט אחד ב קטע...
dtype: object
orig_sent
819    שאלנו על ה פולנים : האם יכלו לעשות יותר מ ש עש...
813    שאלנו את הוא על ה יהודים : האם יכלו לעשות יותר...
dtype: object
orig_sent
1580    ניו יורק ( אי - פי ) .
5468      פיניקס ( אי - פי ) .
dtype: object
orig_sent
1912    ה קהילה ה יהודית ה קובנית ב ארה"ב מארגנת מסע ה...
2314    ה קהילה ה יהודית של יוצאי קובה ב ארה"ב מארגנת ...
dtype: object
orig_sent
1913    מ דיווח ש הגיע אתמול ל משרד ה תיירות ב ירושלים...
2315    מ דיווח ש הגיע אתמול ל משרד ה תיירות ב י

In [1554]:
align_map.head()

,ud,spmrl,shft,shift_change,global_sent_id
0,1,1,0,False,1
1,2,2,0,False,2
2,3,3,0,False,3
3,4,4,0,False,4
4,5,5,0,False,5


In [1560]:
print(list(filter(lambda x: x != 3, {1,2,3})))


[1, 2]


In [1567]:
def filter_sets_dict(sd):
    return {k: set(filter(lambda x: x!=k, v)) for k,v in sd.items()}
all_sim_sp_full_drop_own = filter_sets_dict(all_sim_sp_full)
all_sim_ud_full_drop_own = filter_sets_dict(all_sim_ud_full)

In [1568]:
sentence_align_map = align_map.copy()
sentence_align_map['spmrl_very_similar'] = sentence_align_map.spmrl.map(all_sim_sp_full_drop_own)
sentence_align_map['ud_very_similar'] = sentence_align_map.ud.map(all_sim_ud_full_drop_own)
sentence_align_map.head()

,ud,spmrl,shft,shift_change,global_sent_id,spmrl_very_similar,ud_very_similar
0,1,1,0,False,1,NaN,NaN
1,2,2,0,False,2,NaN,NaN
2,3,3,0,False,3,NaN,NaN
3,4,4,0,False,4,NaN,NaN
4,5,5,0,False,5,NaN,NaN


In [1587]:
def get_dup_sets(sents):
    return [x for x in list(sents.groupby(sents).apply(lambda x: set(x.index))) if len(x)>1]

def get_dup_map(sents):
    dup_sets = get_dup_sets(sents)
    dup_map = {}
    for s in dup_sets:
        for e in s:
            dup_map[e] = s
    return filter_sets_dict(dup_map)

sp_dup_map = get_dup_map(sp_sents)
ud_dup_map = get_dup_map(ud_sents_clean)

In [1588]:
sentence_align_map['spmrl_duplicates'] = sentence_align_map.spmrl.map(sp_dup_map)
sentence_align_map['ud_duplicates'] = sentence_align_map.ud.map(ud_dup_map)

In [1589]:
sentence_align_map[(~sentence_align_map.ud_duplicates.isna())]

,ud,spmrl,shft,shift_change,global_sent_id,spmrl_very_similar,ud_very_similar,spmrl_duplicates,ud_duplicates
610,611,611,0,False,611,NaN,NaN,NaN,{2837}
948,949,949,0,False,949,NaN,NaN,NaN,{4058}
1596,1597,1597,0,False,1597,NaN,NaN,NaN,{1616}
1670,1672,1671,1,False,1671,NaN,NaN,NaN,{5164}
2104,2106,2105,1,False,2105,"{5006, 2287, 2324, 4988, 4927}","{2325, 4941, 5002, 3379, 3549, 5020, 5169, 378...",NaN,"{5169, 3379, 5282}"
2227,2229,2228,1,False,2228,NaN,NaN,NaN,"{3314, 5071, 3749, 5099, 4210}"
2286,2288,2287,1,False,2287,"{5006, 2324, 2105, 4988, 4927}","{2325, 4941, 5002, 3379, 3549, 5020, 5169, 210...",NaN,"{3785, 3549, 2405}"
3295,3299,3296,3,False,3296,NaN,NaN,NaN,"{4069, 3301}"
4114,4126,4115,11,False,4115,NaN,{5432},NaN,{5432}
4118,4130,4119,11,False,4119,NaN,{5436},NaN,{5436}


In [9]:
sentence_align_map = pd.read_csv('align/sentence_align_map.csv')

## 2. Align tokens to morphemes in SPMRL & UD

### SPMRL

1. Use gold lattices

In [838]:
def read_lattices(path):
    df = (pd.read_csv(path, sep='\t', header=None, quoting=3, 
                names = ['ID1', 'ID2', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'TOKEN'])
                # add sentence labels
                .assign(sent = lambda x: (x.ID1==0).cumsum())
               )
    return df


In [839]:
sp_test = read_lattices('hebtb_spmrl/test_hebtb-gold.lattices')
sp_dev = read_lattices('hebtb_spmrl/dev_hebtb-gold.lattices')
sp_train = read_lattices('hebtb_spmrl/train_hebtb-gold.lattices')
sp_dev.sent.nunique(), sp_train.sent.nunique(), sp_test.sent.nunique()

(500, 4937, 716)

In [840]:
sp_dev['orig_sent'] = sp_dev.sent

sp_train['orig_sent'] = sp_train.sent + 500
sp_test['orig_sent'] = sp_test.sent + 500 + 4937

sp_dev.orig_sent.min(), sp_dev.orig_sent.max(), sp_train.orig_sent.min(), sp_train.orig_sent.max(), sp_test.orig_sent.min(), sp_test.orig_sent.max()



(1, 500, 501, 5437, 5438, 6153)

In [841]:
sp_lattices = pd.concat([sp_dev, sp_train, sp_test])
sp_lattices.head()

,ID1,ID2,FORM,LEMMA,UPOS,XPOS,FEATS,TOKEN,sent,orig_sent
0,0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,1
1,1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,1
2,2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1,1
3,3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,1
4,4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,1


In [842]:
def read_tokens(path):
    toks = []
    curr_sent=1
    curr_tok=1
    with open(path, 'r', encoding='utf8') as f:
        for token in f:
            if(token=='\n'):
                curr_sent+=1
                curr_tok=1
            else:
                toks.append((curr_sent, curr_tok, token.strip()))
                curr_tok+=1
            
    toks = pd.DataFrame(toks, columns=['sent', 'TOKEN', 'token_str'])
    return toks

dev_tok = read_tokens('hebtb_spmrl/dev_hebtb-tokens.txt')
train_tok = read_tokens('hebtb_spmrl/train_hebtb-tokens.txt')
test_tok = read_tokens('hebtb_spmrl/test_hebtb-tokens.txt')
dev_tok.sent.nunique(), train_tok.sent.nunique(), test_tok.sent.nunique()

(500, 4937, 716)

In [843]:
dev_tok['orig_sent'] = dev_tok.sent

train_tok['orig_sent'] = train_tok.sent + 500
test_tok['orig_sent'] = test_tok.sent + 500 + 4937

dev_tok.orig_sent.min(), dev_tok.orig_sent.max(), train_tok.orig_sent.min(), train_tok.orig_sent.max(), test_tok.orig_sent.min(), test_tok.orig_sent.max()



(1, 500, 501, 5437, 5438, 6153)

In [844]:
sp_tokens = pd.concat([dev_tok, train_tok, test_tok])
sp_tokens.head()

,sent,TOKEN,token_str,orig_sent
0,1,1,עשרות,1
1,1,2,אנשים,1
2,1,3,מגיעים,1
3,1,4,מתאילנד,1
4,1,5,לישראל,1


In [845]:
sp_lattices = sp_lattices.merge(sp_tokens[['orig_sent', 'TOKEN', 'token_str']], how='left')
sp_lattices.head()

,ID1,ID2,FORM,LEMMA,UPOS,XPOS,FEATS,TOKEN,sent,orig_sent,token_str
0,0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,1,עשרות
1,1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,1,אנשים
2,2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1,1,מגיעים
3,3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,1,מתאילנד
4,4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,1,מתאילנד


### UD

In [1068]:
list(ud_dev[0])[:5]

[OrderedDict([('id', 1),
              ('form', 'עשרות'),
              ('lemma', 'עשרות'),
              ('upostag', 'NUM'),
              ('xpostag', 'NUM'),
              ('feats',
               OrderedDict([('Definite', 'Cons'),
                            ('Gender', 'Fem'),
                            ('Number', 'Plur')])),
              ('head', 2),
              ('deprel', 'nummod'),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', 2),
              ('form', 'אנשים'),
              ('lemma', 'איש'),
              ('upostag', 'NOUN'),
              ('xpostag', 'NOUN'),
              ('feats', OrderedDict([('Gender', 'Masc'), ('Number', 'Plur')])),
              ('head', 3),
              ('deprel', 'nsubj'),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', 3),
              ('form', 'מגיעים'),
              ('lemma', 'הגיע'),
              ('upostag', 'VERB'),
              ('xpostag', 'VERB'),
              ('f

In [847]:
from collections import OrderedDict
def get_conllu_token_morpheme_alignment(tl):
    mt_align = []
    segmented = False
    sent_id = tl.metadata['sent_id']
    tok_id=0
    for t in tl:
        morph_id = t['id']
        if type(morph_id) is tuple:
            segmented=True
            segmented_up_to = morph_id[2]
            token = t['form']
            tok_id+=1
        else:
            if segmented and morph_id>segmented_up_to:
                segmented=False
                token = t['form']
            if not segmented:
                tok_id+=1
                token = t['form']
            mt_align.append((sent_id, morph_id, t['form'], tok_id, token))

    return pd.DataFrame(mt_align, columns=['sent', 'morph_id', 'form', 'TOKEN', 'token_str'])

get_conllu_token_morpheme_alignment(ud_dev[0])
ud_tokens = [get_conllu_token_morpheme_alignment(tl) for tl in ud_dev] + [get_conllu_token_morpheme_alignment(tl) for tl in ud_train] + [get_conllu_token_morpheme_alignment(tl) for tl in ud_test]
ud_tokens = pd.concat(ud_tokens).rename(columns={'sent': 'orig_sent'})
ud_tokens.head(10)

,orig_sent,morph_id,form,TOKEN,token_str
0,1,1,עשרות,1,עשרות
1,1,2,אנשים,2,אנשים
2,1,3,מגיעים,3,מגיעים
3,1,4,מ,4,מתאילנד
4,1,5,תאילנד,4,מתאילנד
5,1,6,ל,5,לישראל
6,1,7,ישראל,5,לישראל
7,1,8,כש,6,כשהם
8,1,9,הם,6,כשהם
9,1,10,נרשמים,7,נרשמים


In [848]:
ud_tokens_only = ud_tokens[['orig_sent', 'TOKEN', 'token_str']].drop_duplicates().reset_index().drop('index', axis=1)
ud_tokens_only.head()

,orig_sent,TOKEN,token_str
0,1,1,עשרות
1,1,2,אנשים
2,1,3,מגיעים
3,1,4,מתאילנד
4,1,5,לישראל


## 2.5. Check tokens align: SPMRL <--> UD

In [849]:
ud_tokens_only['global_sent_id'] = ud_tokens_only.orig_sent.map(align_map.set_index('ud').global_sent_id)
sp_tokens['global_sent_id'] = sp_tokens.orig_sent.map(align_map.set_index('spmrl').global_sent_id)

In [850]:
ud_tokens_only.shape

(115535, 4)

In [851]:
ud_tokens_only[~ud_tokens_only.global_sent_id.isna()].shape

(114776, 4)

In [852]:
sp_tokens.shape

(114776, 5)

In [853]:
sp_for_compare = sp_tokens[['orig_sent', 'TOKEN', 'token_str']].reset_index().drop('index', axis=1)
sp_for_compare.index

RangeIndex(start=0, stop=114776, step=1)

In [854]:
ud_for_compare = ud_tokens_only[~ud_tokens_only.global_sent_id.isna()][['orig_sent', 'TOKEN', 'token_str']].reset_index().drop('index', axis=1)
ud_for_compare.index

RangeIndex(start=0, stop=114776, step=1)

In [855]:
ne = (ud_for_compare != sp_for_compare).any(1)
ne[~ne]

Series([], dtype: bool)

## 3. Align Morphemes: SPMRL <--> UD

### Check segmentation diff

In [856]:
def get_forms(g):
    return pd.Series({'forms': ' '.join(g.form), 'count':len(g)})

In [897]:
sp_for_compare = (sp_lattices
                  .rename(columns={'FORM':'form'})
                  .groupby(['orig_sent', 'TOKEN', 'token_str'])
                  .apply(get_forms)
                  .reset_index())
sp_for_compare.head()

,orig_sent,TOKEN,token_str,forms,count
0,1,1,עשרות,עשרות,1
1,1,2,אנשים,אנשים,1
2,1,3,מגיעים,מגיעים,1
3,1,4,מתאילנד,מ תאילנד,2
4,1,5,לישראל,ל ישראל,2


In [898]:
ud_for_compare = (ud_tokens
                  .assign(orig_sent=lambda x: x.orig_sent.astype(int))
                  .groupby(['orig_sent', 'TOKEN', 'token_str'])
                  .apply(get_forms)
                  .reset_index())
ud_for_compare.head()

,orig_sent,TOKEN,token_str,forms,count
0,1,1,עשרות,עשרות,1
1,1,2,אנשים,אנשים,1
2,1,3,מגיעים,מגיעים,1
3,1,4,מתאילנד,מ תאילנד,2
4,1,5,לישראל,ל ישראל,2


In [899]:
ud_for_compare_stripped = ud_for_compare.assign(forms = forms.str.replace('_', ''))

In [900]:
ud_for_compare['global_sent_id'] = (ud_for_compare.orig_sent
                                    .map(align_map
                                         .assign(ud=lambda x: x.ud.astype(int))
                                         .set_index('ud').global_sent_id)
                                   )
sp_for_compare['global_sent_id'] = (sp_for_compare.orig_sent
                                    .map(align_map
                                         .set_index('spmrl').global_sent_id))

In [901]:
ud_for_compare = ud_for_compare[~ud_for_compare.global_sent_id.isna()].reset_index().drop(['index', 'orig_sent'], axis=1).assign(global_sent_id=lambda x: x.global_sent_id.astype(int))
sp_for_compare = sp_for_compare.drop('orig_sent', axis=1)

In [917]:
ud_for_compare['global_sent_id_token'] = (ud_for_compare.global_sent_id.astype(str) 
                                          + '_' 
                                          + ud_for_compare.TOKEN.astype(str))
ud_for_compare = (ud_for_compare.set_index('global_sent_id_token')
                  .drop(['global_sent_id', 'TOKEN'], axis=1))
ud_for_compare.head()

,token_str,forms,count
global_sent_id_token,,,
1_1,עשרות,עשרות,1
1_2,אנשים,אנשים,1
1_3,מגיעים,מגיעים,1
1_4,מתאילנד,מ תאילנד,2
1_5,לישראל,ל ישראל,2


In [918]:
sp_for_compare['global_sent_id_token'] = sp_for_compare.global_sent_id.astype(str) + '_' + sp_for_compare.TOKEN.astype(str)
sp_for_compare = (sp_for_compare.set_index('global_sent_id_token')
                  .drop(['global_sent_id', 'TOKEN'], axis=1))
sp_for_compare.head()

,token_str,forms,count
global_sent_id_token,,,
1_1,עשרות,עשרות,1
1_2,אנשים,אנשים,1
1_3,מגיעים,מגיעים,1
1_4,מתאילנד,מ תאילנד,2
1_5,לישראל,ל ישראל,2


In [919]:
ne = (ud_for_compare != sp_for_compare).any(1)
ne[ne].shape

(7377,)

In [920]:
ne[~ne].shape

(107399,)

In [921]:
df1=sp_for_compare
df2=ud_for_compare

In [922]:
ne_stacked = (df1 != df2).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
changed.head()

id    col  
3_4   forms    True
3_28  forms    True
5_16  forms    True
      count    True
6_17  forms    True
dtype: bool

In [923]:
difference_locations = np.where(df1 != df2)
changed_from = df1.values[difference_locations]
changed_to = df2.values[difference_locations]
diffs = pd.DataFrame({'sp': changed_from, 'ud': changed_to}, index=changed.index).reset_index()
diffs.col.value_counts()

forms        7377
count        2553
token_str      51
Name: col, dtype: int64

In [924]:
seg_diffs = diffs[diffs.col=='count']
seg_diffs[seg_diffs.ud<seg_diffs.sp]

,id,col,sp,ud
6932,4136_15,count,3,2


In [932]:
diffs[diffs.col=='forms'].to_csv('ud_spmrl_token_forms_diff.csv', index=False)
diffs[diffs.col=='forms'].head()

,id,col,sp,ud
0,3_4,forms,"ח""ך","ח""כ"
1,3_28,forms,ל המ,ל_ _הם
2,5_16,forms,הזמנתם,הזמנה_ _של_ _הם
4,6_17,forms,ל ה עובדים,ל ה_ עובדים
5,8_20,forms,ל היא,ל_ _היא


In [926]:
diffs[diffs.col=='token_str']

,id,col,sp,ud
748,535_2,token_str,דווח,דוווח
2582,1391_32,token_str,"ח""ך","ח""כ"
2958,1587_5,token_str,89,001
2960,1587_6,token_str,100,98
2962,1587_18,token_str,83,011
2964,1587_19,token_str,110,38
3480,1867_15,token_str,"ח""ך","ח""כ"
3489,1870_1,token_str,"ח""ך","ח""כ"
3499,1871_24,token_str,"ח""ך","ח""כ"
3501,1872_1,token_str,"ח""ך","ח""כ"


In [928]:
sp_for_compare.loc['4136_15']

token_str       בהסכם
forms        ב ה הסכם
count               3
Name: 4136_15, dtype: object

In [930]:
ud_for_compare.loc['4136_15']

token_str     בהסכם
forms        ב הסכם
count             2
Name: 4136_15, dtype: object

#### Morpheme alignment

In [1017]:
def get_forms_sp(g):
    return pd.Series({'forms': ' '.join(g.form), 'count':len(g), 'morph_ids': list(g.ID2)})
def get_forms_ud(g):
    return pd.Series({'forms': ' '.join(g.form), 'count':len(g), 'morph_ids': list(g.morph_id)})

In [1043]:
sp_for_compare = (sp_lattices
                  .rename(columns={'FORM':'form'})
                  .groupby(['orig_sent', 'TOKEN', 'token_str'])
                  .apply(get_forms_sp)
                  .reset_index())
sp_for_compare.head()

,orig_sent,TOKEN,token_str,forms,count,morph_ids
0,1,1,עשרות,עשרות,1,[1]
1,1,2,אנשים,אנשים,1,[2]
2,1,3,מגיעים,מגיעים,1,[3]
3,1,4,מתאילנד,מ תאילנד,2,"[4, 5]"
4,1,5,לישראל,ל ישראל,2,"[6, 7]"


In [1044]:
ud_for_compare = (ud_tokens
                  .assign(orig_sent=lambda x: x.orig_sent.astype(int))
                  .groupby(['orig_sent', 'TOKEN', 'token_str'])
                  .apply(get_forms_ud)
                  .reset_index())
ud_for_compare.head()

,orig_sent,TOKEN,token_str,forms,count,morph_ids
0,1,1,עשרות,עשרות,1,[1]
1,1,2,אנשים,אנשים,1,[2]
2,1,3,מגיעים,מגיעים,1,[3]
3,1,4,מתאילנד,מ תאילנד,2,"[4, 5]"
4,1,5,לישראל,ל ישראל,2,"[6, 7]"


In [1045]:
ud_for_compare_stripped = ud_for_compare.assign(forms = lambda x: x.forms.str.replace('_', ''))

In [1046]:
ud_for_compare['global_sent_id'] = (ud_for_compare.orig_sent
                                    .map(align_map
                                         .assign(ud=lambda x: x.ud.astype(int))
                                         .set_index('ud').global_sent_id)
                                   )
sp_for_compare['global_sent_id'] = (sp_for_compare.orig_sent
                                    .map(align_map
                                         .set_index('spmrl').global_sent_id))

In [1047]:
ud_for_compare = ud_for_compare[~ud_for_compare.global_sent_id.isna()].reset_index().drop(['index'], axis=1).assign(global_sent_id=lambda x: x.global_sent_id.astype(int))

In [1048]:
sp_for_compare = sp_for_compare

In [1049]:
sp_for_compare.head()

,orig_sent,TOKEN,token_str,forms,count,morph_ids,global_sent_id
0,1,1,עשרות,עשרות,1,[1],1
1,1,2,אנשים,אנשים,1,[2],1
2,1,3,מגיעים,מגיעים,1,[3],1
3,1,4,מתאילנד,מ תאילנד,2,"[4, 5]",1
4,1,5,לישראל,ל ישראל,2,"[6, 7]",1


## TODO
Alignment
1. ~~Morpheme aligment output~~
1. ~~Binyanim~~
1. ~~BIOSE~~
    1. ~~Evaluate BIOSE~~
1. ~~Fix SPMRL Tokens~~
    1. ~~Change only strings, not segmentation (DONT TOUCH TREE)~~
1. ~~Mark duplicates~~
1. ~~Write CoNLLU~~
1. missing misc/feats from tokens in UD (SpaceAfter)
1. Add text metadata
1. Add duplicate metadata
1. Write function for tag transfer pre-check (using POS dictionary and token_morpheme_id)
1. Document all changes (including
1. Output all sentences with digits (pre and post fixes)

Wikipedia
1. Check different tokenization
1. Run MILA / Add regular expressions

## Morpheme aligment output


In [1410]:
align_df = pd.concat([sp_for_compare[['global_sent_id', 'TOKEN']], sp_for_compare[['orig_sent', 'token_str', 'forms', 'morph_ids', 'count']], ud_for_compare[['orig_sent', 'token_str', 'forms', 'morph_ids', 'count']]], 
                     keys=['global', 'sp', 'ud'], axis=1)
#align_df[['global_sent_id', 'token_id']] = align_df.global_sent_id_token.str.split('_', expand=True)
#flatten
align_df.columns = ['_'.join(col).strip() for col in align_df.columns.values]
align_df.head()

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count
0,1,1,1,עשרות,עשרות,[1],1,1,עשרות,עשרות,[1],1
1,1,2,1,אנשים,אנשים,[2],1,1,אנשים,אנשים,[2],1
2,1,3,1,מגיעים,מגיעים,[3],1,1,מגיעים,מגיעים,[3],1
3,1,4,1,מתאילנד,מ תאילנד,"[4, 5]",2,1,מתאילנד,מ תאילנד,"[4, 5]",2
4,1,5,1,לישראל,ל ישראל,"[6, 7]",2,1,לישראל,ל ישראל,"[6, 7]",2


In [1411]:
align_df.to_csv('align/token_morpheme_alignment_spmrl_ud.csv', index=False)

## Create standard UD and SPMRL DFs

### UD

In [1180]:
ud_dev[0][0]

OrderedDict([('id', 1),
             ('form', 'עשרות'),
             ('lemma', 'עשרות'),
             ('upostag', 'NUM'),
             ('xpostag', 'NUM'),
             ('feats',
              OrderedDict([('Definite', 'Cons'),
                           ('Gender', 'Fem'),
                           ('Number', 'Plur')])),
             ('head', 2),
             ('deprel', 'nummod'),
             ('deps', None),
             ('misc', None)])

In [1181]:
from collections import OrderedDict
def get_conllu_df(cu):
    sents = []
    for sentence in cu:
        tokens = []
        sent_id = sentence.metadata['sent_id']
        #text = sentence.metadata['text']
        for token in sentence:
            if type(token['id'])==int:
                tok = OrderedDict()
                tok['sent_id'] = sent_id
                #tok['text'] = text
                tok.update(token)
                tokens.append(tok)
        sents.extend(tokens)
    return pd.DataFrame(sents)
uddf = (pd.concat([get_conllu_df(ud_dev),
                   get_conllu_df(ud_train),
                   get_conllu_df(ud_test)], axis=0) 
        .merge(align_map[['global_sent_id', 'ud']].rename(columns={'ud': 'sent_id'}), how='inner')
        .merge(ud_tokens[['orig_sent', 'morph_id', 'TOKEN', 'token_str']].rename(columns={'morph_id': 'id', 'TOKEN': 'token_id', 'orig_sent': 'sent_id'}), how='left' )
       )
uddf.tail()

,sent_id,id,form,lemma,upostag,xpostag,feats,head,deprel,deps,misc,global_sent_id,token_id,token_str
160374,6216,5,אך,אך,CCONJ,CCONJ,None,6,cc,None,None,6153,4,אך
160375,6216,6,לא,לא,ADV,ADV,{'Polarity': 'Neg'},4,conj,None,None,6153,5,לא
160376,6216,7,ה,ה,DET,DET,{'PronType': 'Art'},8,det:def,None,None,6153,6,התרופה
160377,6216,8,תרופה,תרופה,NOUN,NOUN,"{'Gender': 'Fem', 'HebSource': 'ConvUncertainH...",6,dep,None,None,6153,6,התרופה
160378,6216,9,.,.,PUNCT,PUNCT,None,4,punct,None,None,6153,7,.


In [1182]:
uddf['token_morph_id'] = uddf.id - uddf.groupby(['global_sent_id', 'token_id'])['id'].transform(min)
uddf.head()

,sent_id,id,form,lemma,upostag,xpostag,feats,head,deprel,deps,misc,global_sent_id,token_id,token_str,token_morph_id
0,1,1,עשרות,עשרות,NUM,NUM,"{'Definite': 'Cons', 'Gender': 'Fem', 'Number'...",2,nummod,None,None,1,1,עשרות,0
1,1,2,אנשים,איש,NOUN,NOUN,"{'Gender': 'Masc', 'Number': 'Plur'}",3,nsubj,None,None,1,2,אנשים,0
2,1,3,מגיעים,הגיע,VERB,VERB,"{'Gender': 'Masc', 'HebBinyan': 'HIFIL', 'Numb...",0,root,None,None,1,3,מגיעים,0
3,1,4,מ,מ,ADP,ADP,None,5,case,None,None,1,4,מתאילנד,0
4,1,5,תאילנד,תאילנד,PROPN,PROPN,None,3,obl,None,None,1,4,מתאילנד,1


### SPMRL

In [1183]:
sp_lattices.head()

,ID1,ID2,FORM,LEMMA,UPOS,XPOS,FEATS,TOKEN,sent,orig_sent,token_str
0,0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,1,עשרות
1,1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,1,אנשים
2,2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1,1,מגיעים
3,3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,1,מתאילנד
4,4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,1,מתאילנד


In [1215]:
spdf = (sp_lattices
        .merge(align_map[['global_sent_id', 'spmrl']].rename(columns={'spmrl': 'orig_sent'}))
        .rename(columns={'orig_sent': 'sent_id', 'TOKEN': 'token_id', 
                         'orig_sent': 'sent_id', 'ID2': 'id', 'UPOS': 'upostag',
                         'XPOS': 'xpostag', 'FORM': 'form', 'LEMMA': 'lemma', 
                         'FEATS': 'feats'})
        .drop(['sent', 'ID1'], axis=1)
       )
spdf.head()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,עשרות,1
1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,אנשים,1
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1,מגיעים,1
3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,מתאילנד,1
4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,מתאילנד,1


In [1216]:
spdf.tail()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id
155325,5,אך,אך,CC,CC,_,4,6153,אך,6153
155326,6,לא,לא,RB,RB,_,5,6153,לא,6153
155327,7,ה,ה,DEF,DEF,_,6,6153,התרופה,6153
155328,8,תרופה,תרופה,NN,NN,gen=F|num=S,6,6153,התרופה,6153
155329,9,.,_,yyDOT,yyDOT,_,7,6153,.,6153


In [1217]:
spdf['token_morph_id'] = spdf.id - spdf.groupby(['global_sent_id', 'token_id'])['id'].transform(min)
spdf.head()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,עשרות,1,0
1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,אנשים,1,0
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1,מגיעים,1,0
3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,מתאילנד,1,0
4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,מתאילנד,1,1


## Make fixes and move HebBinyan

In [1187]:
uddf['binyan'] = uddf.feats.apply(lambda x: x.get('HebBinyan', '---') if x is not None else '---') 

In [1188]:
ud_bin_pos = (uddf.groupby(['global_sent_id', 'token_id'])
              .apply(lambda x: pd.Series({'pos':'|'.join(x.upostag),
                                          'binyan': '|'.join(x.binyan),
                                          'binyan_exists': (x.binyan!='---').any(),
                                          'num_morph': x.shape[0]}))
             )

In [1189]:
ud_bin_pos.head()

pos   binyan  binyan_exists  num_morph
global_sent_id token_id                                              
1              1               NUM      ---          False          1
               2              NOUN      ---          False          1
               3              VERB    HIFIL           True          1
               4         ADP|PROPN  ---|---          False          2
               5         ADP|PROPN  ---|---          False          2

In [1190]:
ud_bin_pos.num_morph.value_counts()

1    77951
2    29537
3     5905
4     1278
5      103
6        2
Name: num_morph, dtype: int64

In [1191]:
ud_bin_pos[(ud_bin_pos.num_morph>1) & (ud_bin_pos.binyan_exists)].pos.value_counts()

SCONJ|VERB          1843
CCONJ|VERB           714
DET|VERB             144
ADP|VERB              68
VERB|ADP|PRON         35
ADP|DET|VERB          26
CCONJ|DET|VERB         7
CCONJ|ADP|VERB         5
CCONJ|SCONJ|VERB       3
SCONJ|DET|VERB         3
CCONJ|PUNCT|VERB       2
SCONJ|ADP|VERB         1
Name: pos, dtype: int64

In [1192]:
sp_pos = (spdf.groupby(['global_sent_id', 'token_id'])
              .apply(lambda x: pd.Series({'pos':'|'.join(x.upostag),
                                          'num_morph': x.shape[0]}))
             )
sp_pos.head()

pos  num_morph
global_sent_id token_id                            
1              1                     CDT          1
               2                      NN          1
               3                      BN          1
               4         PREPOSITION|NNP          2
               5         PREPOSITION|NNP          2

In [1193]:
tok_pos_merge = pd.concat([sp_pos, ud_bin_pos], axis=1, keys=['spmrl', 'ud'])
tok_pos_merge.columns = ['_'.join(col).strip() for col in tok_pos_merge.columns.values]

tok_pos_merge.head()

spmrl_pos  spmrl_num_morph     ud_pos  \
global_sent_id token_id                                                
1              1                     CDT                1        NUM   
               2                      NN                1       NOUN   
               3                      BN                1       VERB   
               4         PREPOSITION|NNP                2  ADP|PROPN   
               5         PREPOSITION|NNP                2  ADP|PROPN   

                        ud_binyan  ud_binyan_exists  ud_num_morph  
global_sent_id token_id                                            
1              1              ---             False             1  
               2              ---             False             1  
               3            HIFIL              True             1  
               4          ---|---             False             2  
               5          ---|---             False             2

In [1194]:
(tok_pos_merge[(tok_pos_merge.spmrl_num_morph>1) & (tok_pos_merge.ud_binyan_exists)]
 .groupby(['spmrl_pos', 'ud_pos']).size().sort_values(ascending=False)
         )

spmrl_pos             ud_pos          
REL|VB                SCONJ|VERB          1040
REL|BN                SCONJ|VERB           763
CONJ|VB               CCONJ|VERB           516
CONJ|BN               CCONJ|VERB           192
DEF|BN                DET|VERB             144
TEMP|VB               SCONJ|VERB            30
PREPOSITION|BN        ADP|VERB              28
PREPOSITION|VB        ADP|VERB              27
PREPOSITION|DEF|BN    ADP|DET|VERB          26
VB|AT|S_PRN           VERB|ADP|PRON         26
PREPOSITION|BNT       ADP|VERB              13
TEMP|BN               SCONJ|VERB             8
CONJ|DEF|BN           CCONJ|DET|VERB         7
CONJ|BNT              CCONJ|VERB             6
VB|AT|S_ANP           VERB|ADP|PRON          3
VB|POS|S_PRN          VERB|ADP|PRON          3
VB|DUMMY_AT|S_ANP     VERB|ADP|PRON          2
CONJ|yyQUOT|VB        CCONJ|PUNCT|VERB       2
CONJ|TEMP|VB          CCONJ|ADP|VERB         2
CONJ|REL|VB           CCONJ|SCONJ|VERB       2
TEMP|DEF|BN          

In [1218]:
spdf = spdf.merge(uddf[['global_sent_id', 'token_id', 'token_morph_id', 'binyan']], how='left')
spdf['binyan'] = spdf.binyan.fillna('---')
spdf.head()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,עשרות,1,0,---
1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,אנשים,1,0,---
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1,מגיעים,1,0,HIFIL
3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,מתאילנד,1,0,---
4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,מתאילנד,1,1,---


In [1593]:
spdf.loc[spdf.binyan!='---', 'feats'] = spdf.feats+'|HebBinyan='+spdf.binyan
spdf.head()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,עשרות,1,0,---,O,0,_
1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,אנשים,1,0,---,O,0,_
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A|HebBinyan=HIFIL,3,1,מגיעים,1,0,HIFIL,O,0,_
3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,מתאילנד,1,0,---,O,0,_
4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,מתאילנד,1,1,---,S-GPE,1,GPE


## Move NER to SPMRL

In [1467]:
def get_orig_sent(s):
    start = int(s.split('-')[0].split('_')[1])
    sent = int(s.split('.')[1].split('_')[1])
    orig_sent = start+sent-1
    return orig_sent

ner = (pd.read_csv('data/curation.csv.gz')
        .assign(sent=lambda x: x.file+'_'+x.sent.astype(str))
        .assign(sent_id = lambda x: x.sent.apply(get_orig_sent).astype(str))
        .assign(id = lambda x: x.sent_tok_num.str.split('-', expand=True)[1].astype(int))
        .merge(align_map[['global_sent_id', 'ud']].rename(columns={'ud': 'sent_id'}), how='inner')
        .merge(ud_tokens[['orig_sent', 'morph_id', 'TOKEN', 'token_str']]
               .rename(columns={'morph_id': 'id', 'TOKEN': 'token_id', 'orig_sent': 'sent_id'}), how='left' )
        )
ner['token_morph_id'] = ner.id - ner.groupby(['global_sent_id', 'token_id'])['id'].transform(min)
ner.head()


,sent_tok_num,tok_offset,token,FEAT_gender,FEAT_number,FEAT_case,FEAT_degree,FEAT_transitivity,FEAT_tense,FEAT_mood,...,ner_type,is_ner,biose,file,sent_id,id,global_sent_id,token_id,token_str,token_morph_id
0,1-1,0-5,עשרות,Fem,Plur,*,*,*,*,*,...,_,False,O,dev_1-100.tsv,1,1,1,1,עשרות,0
1,1-2,6-11,אנשים,Masc,Plur,*,*,*,*,*,...,_,False,O,dev_1-100.tsv,1,2,1,2,אנשים,0
2,1-3,12-18,מגיעים,Masc,Plur,*,*,*,*,*,...,_,False,O,dev_1-100.tsv,1,3,1,3,מגיעים,0
3,1-4,19-20,מ,_,_,_,_,_,_,_,...,_,False,O,dev_1-100.tsv,1,4,1,4,מתאילנד,0
4,1-5,21-27,תאילנד,_,_,_,_,_,_,_,...,GPE,True,S-GPE,dev_1-100.tsv,1,5,1,4,מתאילנד,1


In [1468]:
ner.head().T

,0,1,2,3,4
sent_tok_num,1-1,1-2,1-3,1-4,1-5
tok_offset,0-5,6-11,12-18,19-20,21-27
token,עשרות,אנשים,מגיעים,מ,תאילנד
FEAT_gender,Fem,Masc,Masc,_,_
FEAT_number,Plur,Plur,Plur,_,_
FEAT_case,*,*,*,_,_
FEAT_degree,*,*,*,_,_
FEAT_transitivity,*,*,*,_,_
FEAT_tense,*,*,*,_,_
FEAT_mood,*,*,*,_,_


In [1469]:
spdf = (spdf.merge(ner[['global_sent_id', 'token_id', 'token_morph_id', 'biose', 'ner', 'ner_layers']], how='left')
       )

In [1470]:
spdf.groupby(['global_sent_id', 'token_id']).id.nunique().value_counts()

1    79287
2    30614
3     4687
4      186
5        2
Name: id, dtype: int64

In [1471]:
spdf.token_morph_id.value_counts()

0    114776
1     35489
2      4875
3       188
4         2
Name: token_morph_id, dtype: int64

In [1472]:
spdf['biose'] = spdf.biose.fillna('O')
spdf['ner_layers'] = spdf.ner_layers.fillna(0).astype(int)
spdf['ner'] = spdf.ner.fillna('_')


In [1473]:
spdf[['global_sent_id', 'token_id','token_morph_id', 'biose', 'token_str', 'form']].to_csv('data/spdf.csv', index=False)
ner[['global_sent_id', 'token_id','token_morph_id', 'biose', 'token_str', 'token']].to_csv('data/ner.csv', index=False)


In [1474]:
uddf = (uddf.merge(ner[['global_sent_id', 'token_id', 'token_morph_id', 'biose', 'ner', 'ner_layers']], how='left')
       )
uddf['biose'] = uddf.biose.fillna('O')
uddf['ner_layers'] = uddf.ner_layers.fillna(0).astype(int)
uddf['ner'] = uddf.ner.fillna('_')


In [1475]:
uddf.head()

,sent_id,id,form,lemma,upostag,xpostag,feats,head,deprel,deps,misc,global_sent_id,token_id,token_str,token_morph_id,binyan,biose,ner,ner_layers
0,1,1,עשרות,עשרות,NUM,NUM,"{'Definite': 'Cons', 'Gender': 'Fem', 'Number'...",2,nummod,None,None,1,1,עשרות,0,---,O,_,0
1,1,2,אנשים,איש,NOUN,NOUN,"{'Gender': 'Masc', 'Number': 'Plur'}",3,nsubj,None,None,1,2,אנשים,0,---,O,_,0
2,1,3,מגיעים,הגיע,VERB,VERB,"{'Gender': 'Masc', 'HebBinyan': 'HIFIL', 'Numb...",0,root,None,None,1,3,מגיעים,0,HIFIL,O,_,0
3,1,4,מ,מ,ADP,ADP,None,5,case,None,None,1,4,מתאילנד,0,---,O,_,0
4,1,5,תאילנד,תאילנד,PROPN,PROPN,None,3,obl,None,None,1,4,מתאילנד,1,---,S-GPE,GPE,1


In [1476]:
spdf.head()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,עשרות,1,0,---,O,0,_
1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,אנשים,1,0,---,O,0,_
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1,מגיעים,1,0,HIFIL,O,0,_
3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,מתאילנד,1,0,---,O,0,_
4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,מתאילנד,1,1,---,S-GPE,1,GPE


In [1477]:
spdf.tail(70)

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner
155260,20,חומץ,חומץ,NN,NN,gen=M|num=S,15,6149,חומץ,6149,0,---,O,0,_
155261,21,",",_,yyCM,yyCM,_,16,6149,",",6149,0,---,O,0,_
155262,22,עשבי,עשב,NNT,NNT,gen=M|num=P,17,6149,עשבי,6149,0,---,O,0,_
155263,23,תיבול,תיבול,NN,NN,gen=M|num=S,18,6149,תיבול,6149,0,---,O,0,_
155264,24,ו,ו,CONJ,CONJ,_,19,6149,ותבלינים,6149,0,---,O,0,_
155265,25,תבלינים,תבלין,NN,NN,gen=M|num=P,19,6149,ותבלינים,6149,1,---,O,0,_
155266,26,.,_,yyDOT,yyDOT,_,20,6149,.,6149,0,---,O,0,_
155267,1,אף,אף,CC,CC,_,1,6150,אף,6150,0,---,O,0,_
155268,2,ש,ש,REL,REL,_,2,6150,שהמנה,6150,0,---,O,0,_
155269,3,ה,_,DEF,DEF,_,2,6150,שהמנה,6150,1,---,O,0,_


In [1478]:
spdf.shape[0]

155330

In [1479]:
uddf.shape[0]

160379

## Fix Tokens

In [1506]:
afix = pd.read_csv('align/token_morpheme_alignment_spmrl_ud_with_fixes.csv')
afix.head()

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count,fixed_token_str,fixed_sp_forms,fixed_ud_forms,comment
0,1,1,1,עשרות,עשרות,[1],1,1,עשרות,עשרות,[1],1,NaN,NaN,NaN,NaN
1,1,2,1,אנשים,אנשים,[2],1,1,אנשים,אנשים,[2],1,NaN,NaN,NaN,NaN
2,1,3,1,מגיעים,מגיעים,[3],1,1,מגיעים,מגיעים,[3],1,NaN,NaN,NaN,NaN
3,1,4,1,מתאילנד,מ תאילנד,"[4, 5]",2,1,מתאילנד,מ תאילנד,"[4, 5]",2,NaN,NaN,NaN,NaN
4,1,5,1,לישראל,ל ישראל,"[6, 7]",2,1,לישראל,ל ישראל,"[6, 7]",2,NaN,NaN,NaN,NaN


In [1507]:
afix[(afix.fixed_token_str.isna()) & (afix.sp_token_str.str.startswith('0')) & (afix.sp_token_str!='0')]

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count,fixed_token_str,fixed_sp_forms,fixed_ud_forms,comment
971,50,28,50,0192,0192,[36],1,50,0192,0192,[36],1,NaN,NaN,NaN,NaN
60926,3261,13,3261,04833280,04833280,[16],1,3264,04833280,04833280,[16],1,NaN,NaN,NaN,NaN
90624,4863,5,4863,01.00,01.00,[7],1,4877,01.00,01.00,[7],1,NaN,NaN,NaN,NaN


In [1508]:
afix[afix.sp_count.isna()]

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count,fixed_token_str,fixed_sp_forms,fixed_ud_forms,comment


In [1509]:
afix[afix.comment=='קפ'].shape[0]

39

In [1510]:
afix.loc[(~(afix.comment=='קפ')) & (~afix.fixed_token_str.isna()) & (afix.fixed_sp_forms.isna()) & (afix.fixed_ud_forms.isna()), 'fixed_sp_forms'] = afix[(~(afix.comment=='קפ')) & (~afix.fixed_token_str.isna()) & (afix.fixed_sp_forms.isna()) & (afix.fixed_ud_forms.isna())]['fixed_token_str']


In [1511]:
afix.loc[(~(afix.comment=='קפ')) & (~afix.fixed_token_str.isna()) & (~afix.fixed_sp_forms.isna()) & (afix.fixed_ud_forms.isna()), 'fixed_ud_forms'] = afix[(~(afix.comment=='קפ')) & (~afix.fixed_token_str.isna()) & (~afix.fixed_sp_forms.isna()) & (afix.fixed_ud_forms.isna())]['fixed_sp_forms']

In [1512]:
afix[afix.fixed_sp_forms.fillna('').astype(str).apply(lambda x: len(x.split()))!=afix.fixed_ud_forms.fillna('').astype(str).apply(lambda x: len(x.split()))]

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count,fixed_token_str,fixed_sp_forms,fixed_ud_forms,comment
47334,2485,1,2485,"קפספבינהמנכ""ל","קפספבין ה מנכ""ל","[1, 2, 3]",3,2487,"קפספבינהמנכ""ל","קפספבין ה מנכ""ל","[1, 2, 3]",3,המנכ”ל,ה מנכ”ל,NaN,קפ
49885,2645,1,2645,קפספבינהערב,קפספבין הערב,"[1, 2]",2,2647,קפספבינהערב,קפספבין הערב,"[1, 2]",2,הערב,ה ערב,NaN,קפ
63150,3383,1,3383,קפידבינהחוקרים,קפידבין ה חוקרים,"[1, 2, 3]",3,3389,קפידבינהחוקרים,קפידבין ה חוקרים,"[1, 2, 3]",3,החוקרים,ה חוקרים,NaN,קפ
69767,3749,1,3749,קפידבינבסניף,קפידבין ב סניף,"[1, 2, 3]",3,3757,קפידבינבסניף,קפידבין ב סניף,"[1, 2, 3]",3,בסניף,ב סניף,NaN,קפ
70053,3765,1,3765,קפידבינהתובעת,קפידבין ה תובעת,"[1, 2, 3]",3,3773,קפידבינהתובעת,קפידבין ה תובעת,"[1, 2, 3]",3,התובעת,ה תובעת,NaN,קפ
80680,4387,1,4387,קפידבינהמקורות,קפידבין ה מקורות,"[1, 2, 3]",3,4400,קפידבינהמקורות,קפידבין ה מקורות,"[1, 2, 3]",3,המקורות,ה מקורות,NaN,קפ
87032,4674,1,4674,קפידבינהתוכנית,קפידבין ה תוכנית,"[1, 2, 3]",3,4687,קפידבינהתוכנית,קפידבין ה תוכנית,"[1, 2, 3]",3,התוכנית,ה תוכנית,NaN,קפ
90260,4844,1,4844,קפידבינהמלך,קפידבין ה מלך,"[1, 2, 3]",3,4858,קפידבינהמלך,קפידבין ה מלך,"[1, 2, 3]",3,המלך,ה מלך,NaN,קפ
91012,4880,4,4880,כיקפידבינההסתדרות,כי קפידבין ה הסתדרות,"[4, 5, 6, 7]",4,4894,כיקפידבינההסתדרות,כי קפידבין ה הסתדרות,"[4, 5, 6, 7]",4,ההסתדרות,ה הסתדרות,NaN,קפ
91593,4905,1,4905,קפידבינהעיתון,קפידבין ה עיתון,"[1, 2, 3]",3,4919,קפידבינהעיתון,קפידבין ה עיתון,"[1, 2, 3]",3,העיתון,ה עיתון,NaN,קפ


In [1513]:
afix[(~afix.fixed_sp_forms.isna()) & (afix.fixed_sp_forms.fillna('').astype(str).apply(lambda x: len(x.split()))!=afix.sp_count)]

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count,fixed_token_str,fixed_sp_forms,fixed_ud_forms,comment
47334,2485,1,2485,"קפספבינהמנכ""ל","קפספבין ה מנכ""ל","[1, 2, 3]",3,2487,"קפספבינהמנכ""ל","קפספבין ה מנכ""ל","[1, 2, 3]",3,המנכ”ל,ה מנכ”ל,NaN,קפ
63150,3383,1,3383,קפידבינהחוקרים,קפידבין ה חוקרים,"[1, 2, 3]",3,3389,קפידבינהחוקרים,קפידבין ה חוקרים,"[1, 2, 3]",3,החוקרים,ה חוקרים,NaN,קפ
69767,3749,1,3749,קפידבינבסניף,קפידבין ב סניף,"[1, 2, 3]",3,3757,קפידבינבסניף,קפידבין ב סניף,"[1, 2, 3]",3,בסניף,ב סניף,NaN,קפ
70053,3765,1,3765,קפידבינהתובעת,קפידבין ה תובעת,"[1, 2, 3]",3,3773,קפידבינהתובעת,קפידבין ה תובעת,"[1, 2, 3]",3,התובעת,ה תובעת,NaN,קפ
80680,4387,1,4387,קפידבינהמקורות,קפידבין ה מקורות,"[1, 2, 3]",3,4400,קפידבינהמקורות,קפידבין ה מקורות,"[1, 2, 3]",3,המקורות,ה מקורות,NaN,קפ
87032,4674,1,4674,קפידבינהתוכנית,קפידבין ה תוכנית,"[1, 2, 3]",3,4687,קפידבינהתוכנית,קפידבין ה תוכנית,"[1, 2, 3]",3,התוכנית,ה תוכנית,NaN,קפ
90260,4844,1,4844,קפידבינהמלך,קפידבין ה מלך,"[1, 2, 3]",3,4858,קפידבינהמלך,קפידבין ה מלך,"[1, 2, 3]",3,המלך,ה מלך,NaN,קפ
91012,4880,4,4880,כיקפידבינההסתדרות,כי קפידבין ה הסתדרות,"[4, 5, 6, 7]",4,4894,כיקפידבינההסתדרות,כי קפידבין ה הסתדרות,"[4, 5, 6, 7]",4,ההסתדרות,ה הסתדרות,NaN,קפ
91593,4905,1,4905,קפידבינהעיתון,קפידבין ה עיתון,"[1, 2, 3]",3,4919,קפידבינהעיתון,קפידבין ה עיתון,"[1, 2, 3]",3,העיתון,ה עיתון,NaN,קפ
100086,5333,1,5333,קפידבינברצועה,קפידבין ב ה רצועה,"[1, 2, 3, 4]",4,5352,קפידבינברצועה,קפידבין ב ה_ רצועה,"[1, 2, 3, 4]",4,ברצועה,ב ה רצועה,ב ה_ רצועה,קפ


In [1514]:
afix[(~afix.fixed_ud_forms.isna()) & (afix.fixed_ud_forms.fillna('').astype(str).apply(lambda x: len(x.split()))!=afix.ud_count)]

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count,fixed_token_str,fixed_sp_forms,fixed_ud_forms,comment
100086,5333,1,5333,קפידבינברצועה,קפידבין ב ה רצועה,"[1, 2, 3, 4]",4,5352,קפידבינברצועה,קפידבין ב ה_ רצועה,"[1, 2, 3, 4]",4,ברצועה,ב ה רצועה,ב ה_ רצועה,קפ


In [1515]:
afix.head()

,global_global_sent_id,global_TOKEN,sp_orig_sent,sp_token_str,sp_forms,sp_morph_ids,sp_count,ud_orig_sent,ud_token_str,ud_forms,ud_morph_ids,ud_count,fixed_token_str,fixed_sp_forms,fixed_ud_forms,comment
0,1,1,1,עשרות,עשרות,[1],1,1,עשרות,עשרות,[1],1,NaN,NaN,NaN,NaN
1,1,2,1,אנשים,אנשים,[2],1,1,אנשים,אנשים,[2],1,NaN,NaN,NaN,NaN
2,1,3,1,מגיעים,מגיעים,[3],1,1,מגיעים,מגיעים,[3],1,NaN,NaN,NaN,NaN
3,1,4,1,מתאילנד,מ תאילנד,"[4, 5]",2,1,מתאילנד,מ תאילנד,"[4, 5]",2,NaN,NaN,NaN,NaN
4,1,5,1,לישראל,ל ישראל,"[6, 7]",2,1,לישראל,ל ישראל,"[6, 7]",2,NaN,NaN,NaN,NaN


In [1516]:
spdf_fixed = spdf.copy()
uddf_fixed = uddf.copy()
def fix_conll_tokens(spdf, uddf, afix, filter_comments = ['קפ', ], fix_lemma=True):
    for i, row in afix.loc[(~(afix.comment.isin(filter_comments))) & (~afix.fixed_token_str.isna())].iterrows():
        #spdf.loc[(spdf.global_sent_id==(row['global_global_sent_id']) & (spdf.id==)]
        sp_morph_ids = eval(row['sp_morph_ids'])
        fixed_sp_forms = row['fixed_sp_forms'].split()
        for mid, fsp in zip(sp_morph_ids, fixed_sp_forms):
            spdf.loc[(spdf.global_sent_id==(row['global_global_sent_id'])) & (spdf.id==mid), 'form'] = fsp
            spdf.loc[(spdf.global_sent_id==(row['global_global_sent_id'])) & (spdf.id==mid), 'token_str'] = row['fixed_token_str']
            if (fix_lemma and  (spdf.loc[(spdf.global_sent_id==(row['global_global_sent_id']))
                                         & (spdf.id==mid), 'lemma'].values[0]!='_')):
                spdf.loc[(spdf.global_sent_id==(row['global_global_sent_id'])) & (spdf.id==mid), 'lemma'] = fsp


        ud_morph_ids = eval(row['ud_morph_ids'])
        fixed_ud_forms = row['fixed_ud_forms'].split()
        for mid, fud in zip(ud_morph_ids, fixed_ud_forms):
            uddf.loc[(uddf.global_sent_id==(row['global_global_sent_id'])) & (uddf.id==mid), 'form'] = fud
            uddf.loc[(uddf.global_sent_id==(row['global_global_sent_id'])) & (uddf.id==mid), 'token_str'] = row['fixed_token_str']
            if (fix_lemma and  (uddf.loc[(uddf.global_sent_id==(row['global_global_sent_id'])) 
                                         & (uddf.id==mid), 'lemma'].values[0]!='_')):
                uddf.loc[(uddf.global_sent_id==(row['global_global_sent_id'])) & (uddf.id==mid), 'lemma'] = fud
                
fix_conll_tokens(spdf_fixed, uddf_fixed, afix)
    
spdf_fixed[spdf_fixed.form!=spdf.form]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner
47,5,"ח""כ","ח""כ",NN,NN,gen=M|num=S,4,3,"ח""כ",3,0,---,O,0,_
305,1,"ח""כ","ח""כ",NN,NN,gen=M|num=S,1,11,"ח""כ",11,0,---,O,0,_
348,1,"ח""כ","ח""כ",NN,NN,gen=M|num=S,1,13,"ח""כ",13,0,---,O,0,_
423,1,"ח""כ","ח""כ",NN,NN,gen=M|num=S,1,15,"ח""כ",15,0,---,O,0,_
2266,2,1980,_,CD,CD,_,1,83,ב1980,83,1,---,O,0,_
2500,19,1988,_,CD,CD,_,15,94,מ1988,94,1,---,O,0,_
2935,9,200,_,CD,CD,_,8,110,200,110,0,---,O,0,_
2953,4,25,_,CD,CD,_,4,111,25,111,0,---,O,0,_
5022,3,1977,_,CD,CD,_,2,205,1977,205,0,---,O,0,_
6015,21,1989,_,CD,CD,_,16,240,ב1989,240,1,---,O,0,_


In [1517]:
spdf_fixed[(spdf_fixed.form!=spdf.form) & (spdf_fixed.lemma!='_')]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner
47,5,"ח""כ","ח""כ",NN,NN,gen=M|num=S,4,3,"ח""כ",3,0,---,O,0,_
305,1,"ח""כ","ח""כ",NN,NN,gen=M|num=S,1,11,"ח""כ",11,0,---,O,0,_
348,1,"ח""כ","ח""כ",NN,NN,gen=M|num=S,1,13,"ח""כ",13,0,---,O,0,_
423,1,"ח""כ","ח""כ",NN,NN,gen=M|num=S,1,15,"ח""כ",15,0,---,O,0,_
34244,42,"ח""כ","ח""כ",NN,NN,gen=M|num=S,32,1391,"ח""כ",1391,0,---,O,0,_
47409,23,"ח""כ","ח""כ",NN,NN,gen=M|num=S,15,1867,"ח""כ",1867,0,---,O,0,_
47496,15,"ח""כ","ח""כ",NN,NN,gen=M|num=S,12,1869,הח”כ,1869,1,---,O,0,_
47508,1,"ח""כ","ח""כ",NN,NN,gen=M|num=S,1,1870,"ח""כ",1870,0,---,O,0,_
47542,35,"ח""כ","ח""כ",NN,NN,gen=M|num=S,26,1870,הח”כ,1870,1,---,O,0,_
47592,34,"ח""כ","ח""כ",NN,NN,gen=M|num=S,24,1871,"ח""כ",1871,0,---,O,0,_


### Evaluate Named Entities

In [1518]:
import importlib
importlib.reload(nem)

<module 'ne_evaluate_mentions' from '/home/dan/NER/ne_evaluate_mentions.py'>

In [1519]:
import ne_evaluate_mentions as nem

In [1519]:
sp_sents_for_ment = spdf_fixed.assign(token = lambda x: x.form.str.strip('_')).groupby('global_sent_id')[['form', 'biose']].apply(lambda x: x.values.tolist())
sp_mentions = nem.sents_to_mentions(sp_sents_for_ment, None)
sp_mentions[:5]

[(1, 'תאילנד', 'GPE', 1),
 (1, 'ישראל', 'GPE', 1),
 (2, 'וועדת ה עבודה ו ה רווחה', 'ORG', 1),
 (2, 'ה כנסת', 'ORG', 1),
 (3, 'אורה נמיר', 'PER', 1)]

In [1520]:
ud_sents_for_ment = uddf_fixed.assign(token = lambda x: x.form.str.strip('_')).groupby('global_sent_id')[['token', 'biose']].apply(lambda x: x.values.tolist())
ud_mentions = nem.sents_to_mentions(ud_sents_for_ment, None)
ud_mentions[:5]

[(1, 'תאילנד', 'GPE', 1),
 (1, 'ישראל', 'GPE', 1),
 (2, 'וועדת ה עבודה ו ה רווחה', 'ORG', 1),
 (2, 'ה כנסת', 'ORG', 1),
 (3, 'אורה נמיר', 'PER', 1)]

In [1521]:
nem.evaluate_mentions(ud_mentions, sp_mentions, examples=20)

7711 mentions, 7705 found, 7703 correct.
Precision: 1.0
Recall:    1.0
F1:        1.0
FP ex.: ['חייו ה אמיתיים של אלחנדרו מייטה', 'ה ברוקרים של ה רעיונות : צוותות חשיבה ו עלייתה של עלית מדיניות חדשה']
FN ex.: ['ב יום מוות של הוא', 'ענף עץ ו מוצר של הוא', 'ב יום מוות של הוא', 'ספוטניק', 'ל אריס של היא', 'חיים של הוא ה אמיתיים של אלחנדרו מייטה', 'ה ברוקרים של ה רעיונות : צוותות חשיבה ו עלייה של היא של עלית מדיניות חדשה', 'מכון ון - ליר']


(0.9989625210737907, 0.999740428293316, 0.999351323300467)

In [1825]:
spdf_fixed.loc[(spdf_fixed.global_sent_id==3658) & (spdf_fixed.id==9), 'biose'] = 'E-ORG'
spdf_fixed.loc[(spdf_fixed.global_sent_id==5610) & (spdf_fixed.id==15), 'biose'] = 'E-ORG'
spdf_fixed.loc[(spdf_fixed.global_sent_id==4116) & (spdf_fixed.id==48), 'biose'] = 'S-DUC'
spdf_fixed.loc[(spdf_fixed.global_sent_id==4121) & (spdf_fixed.id==9), 'biose'] = 'B-ORG'
spdf_fixed.loc[(spdf_fixed.global_sent_id==1096) & (spdf_fixed.id==10), 'biose'] = 'E-WOA'
spdf_fixed.loc[(spdf_fixed.global_sent_id==1204) & (spdf_fixed.id==16), 'biose'] = 'E-WOA'


In [1826]:
sp_sents_for_ment = spdf_fixed.assign(token = lambda x: x.form.str.strip('_')).groupby('global_sent_id')[['form', 'biose']].apply(lambda x: x.values.tolist())
sp_mentions = nem.sents_to_mentions(sp_sents_for_ment, None)
sp_mentions[:5]

[(1, 'תאילנד', 'GPE', 1),
 (1, 'ישראל', 'GPE', 1),
 (2, 'וועדת ה עבודה ו ה רווחה', 'ORG', 1),
 (2, 'ה כנסת', 'ORG', 1),
 (3, 'אורה נמיר', 'PER', 1)]

In [1827]:
ud_sents_for_ment = uddf_fixed.assign(token = lambda x: x.form.str.strip('_')).groupby('global_sent_id')[['token', 'biose']].apply(lambda x: x.values.tolist())
ud_mentions = nem.sents_to_mentions(ud_sents_for_ment, None)
ud_mentions[:5]

[(1, 'תאילנד', 'GPE', 1),
 (1, 'ישראל', 'GPE', 1),
 (2, 'וועדת ה עבודה ו ה רווחה', 'ORG', 1),
 (2, 'ה כנסת', 'ORG', 1),
 (3, 'אורה נמיר', 'PER', 1)]

In [1828]:
nem.evaluate_mentions(ud_mentions, sp_mentions, examples=20)

7711 mentions, 7711 found, 7705 correct.
Precision: 1.0
Recall:    1.0
F1:        1.0
FP ex.: ['חייו ה אמיתיים של אלחנדרו מייטה', 'ענף עץ ו מוצריו', 'ב יום מותו', 'ב יום מותו', 'ל אריסה', 'ה ברוקרים של ה רעיונות : צוותות חשיבה ו עלייתה של עלית מדיניות חדשה']
FN ex.: ['ב יום מוות של הוא', 'ענף עץ ו מוצר של הוא', 'ב יום מוות של הוא', 'חיים של הוא ה אמיתיים של אלחנדרו מייטה', 'ל אריס של היא', 'ה ברוקרים של ה רעיונות : צוותות חשיבה ו עלייה של היא של עלית מדיניות חדשה']


(0.9992218908053431, 0.9992218908053431, 0.9992218908053431)

## Output CONLL / U

    ID: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
    FORM: Word form or punctuation symbol.
    LEMMA: Lemma or stem of word form.
    UPOS: Universal part-of-speech tag.
    XPOS: Language-specific part-of-speech tag; underscore if not available.
    FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
    HEAD: Head of the current word, which is either a value of ID or zero (0).
    DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
    DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
    MISC: Any other annotation.


[TokenList<עשרות, אנשים, מגיעים, מתאילנד, מ, תאילנד, לישראל, ל, ישראל, כשהם, כש, הם, נרשמים, כמתנדבים, כ, מתנדבים, ,, אך, למעשה, משמשים, עובדים, שכירים, זולים, .>,
 TokenList<תופעה, זו, התבררה, אתמול, בוועדת, ב, וועדת, העבודה, ה, עבודה, והרווחה, ו, ה, רווחה, של, הכנסת, ה, כנסת, ,, שדנה, ש, דנה, בנושא, ב, נושא, העסקת, עובדים, זרים, .>,
 TokenList<יו"ר, הוועדה, ה, וועדה, ,, ח"כ, אורה, נמיר, (, מערך, ), ,, טענה, כי, ", מביאים, עובדים, זרים, לישראל, ל, ישראל, על, תקן, של, מתנדבים, מתאילנד, מ, תאילנד, ,, רק, כדי, לא, לשלם, להם, ל_, _הם, שכר, מינימום, .>,
 TokenList<מצד, מ, צד, אחד, רוצה, האוצר, ה, אוצר, להוריד, את, שכר, המינימום, ה, מינימום, ,, ומצד, ו, מ, צד, שני, מתיר, משרד, העבודה, ה, עבודה, והרווחה, ו, ה, רווחה, להעסיק, עובדים, זרים, בפחות, ב, פחות, משכר, מ, שכר, זה, ", .>,
 TokenList<נמיר, הודיעה, כי, תפנה, לשרי, ל, שרי, הפנים, ה, פנים, והעבודה, ו, ה, עבודה, והרווחה, ו, ה, רווחה, ולמזכיר, ו, ל, מזכיר, תנועת, המושבים, ה, מושבים, ,, בתביעה, ב, תביעה, לבטל, את, הזמנתם, הזמנה_, _של_, _הם, 

In [1621]:
sp_test_c = make_conll_df('hebtb_spmrl/test_hebtb-gold.conll')
sp_dev_c = make_conll_df('hebtb_spmrl/dev_hebtb-gold.conll')
sp_train_c = make_conll_df('hebtb_spmrl/train_hebtb-gold.conll')
sp_dev_c['orig_sent'] = sp_dev_c.sent
sp_train_c['orig_sent'] = sp_train_c.sent + 500
sp_test_c['orig_sent'] = sp_test_c.sent + 500 + 4937
sp_conll = pd.concat([sp_dev_c, sp_train_c, sp_test_c]).reset_index().drop('index', axis=1)
sp_conll.head()

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent,orig_sent
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,2,num,_,_,1,1
1,2,אנשים,איש,NN,NN,gen=M|num=P,3,subj,_,_,1,1
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,0,ROOT,_,_,1,1
3,4,מ,מ,PREPOSITION,PREPOSITION,_,3,prepmod,_,_,1,1
4,5,תאילנד,תאילנד,NNP,NNP,_,4,pobj,_,_,1,1


In [1622]:
sp_conll.shape

(155330, 12)

In [1623]:
spdf_fixed.shape

(155330, 15)

In [1624]:
spdf_fixed[['head', 'deprel', 'deps', 'misc']] = sp_conll[['HEAD', 'DEPREL', 'DEPS', 'MISC']]

In [1687]:
list(ud_dev[0])[:5]

[OrderedDict([('id', 1),
              ('form', 'עשרות'),
              ('lemma', 'עשרות'),
              ('upostag', 'NUM'),
              ('xpostag', 'NUM'),
              ('feats',
               OrderedDict([('Definite', 'Cons'),
                            ('Gender', 'Fem'),
                            ('Number', 'Plur')])),
              ('head', 2),
              ('deprel', 'nummod'),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', 2),
              ('form', 'אנשים'),
              ('lemma', 'איש'),
              ('upostag', 'NOUN'),
              ('xpostag', 'NOUN'),
              ('feats', OrderedDict([('Gender', 'Masc'), ('Number', 'Plur')])),
              ('head', 3),
              ('deprel', 'nsubj'),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', 3),
              ('form', 'מגיעים'),
              ('lemma', 'הגיע'),
              ('upostag', 'VERB'),
              ('xpostag', 'VERB'),
              ('f

In [1625]:
spdf_fixed.head()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,עשרות,1,0,---,O,0,_,2,num,_,_
1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,אנשים,1,0,---,O,0,_,3,subj,_,_
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A|HebBinyan=HIFIL,3,1,מגיעים,1,0,HIFIL,O,0,_,0,ROOT,_,_
3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,מתאילנד,1,0,---,O,0,_,3,prepmod,_,_
4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,מתאילנד,1,1,---,S-GPE,1,GPE,4,pobj,_,_


In [1734]:
spdf_fixed.loc[spdf_fixed.feats.isin(['8','2'])]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped
56024,2,פחות,פחותJJT,JJT,gen=M|num=S,8,1,2138,קפידבינפחות,2138,1,---,O,0,_,dep,_,_,NaN,_
135629,4,רצועה,רצועה,NN,NN gen=F|num=S,2,1,5333,קפידבינברצועה,5333,3,---,E-LOC,1,LOC[49],pobj,_,_,NaN,LOC[49]


In [1729]:
spdf_fixed[spdf_fixed.global_sent_id==2138]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped
56023,1,קפידבין,_,ZVL,ZVL,_,1,2138,קפידבינפחות,2138,0,---,O,0,_,11,dep,_,_,_
56024,2,פחות,פחותJJT,JJT,gen=M|num=S,8,1,2138,קפידבינפחות,2138,1,---,O,0,_,dep,_,_,NaN,_
56025,3,מ,מ,PREPOSITION,PREPOSITION,_,2,2138,ממחצית,2138,0,---,O,0,_,2,dep,_,_,_
56026,4,מחצית,_,DTT,DTT,_,2,2138,ממחצית,2138,1,---,O,0,_,8,dep,_,_,_
56027,5,מתוך,מתוך,IN,IN,_,3,2138,מתוך,2138,0,---,O,0,_,8,dep,_,_,_
56028,6,כלל,כלל,DTT,DTT,_,4,2138,כלל,2138,0,---,O,0,_,8,det,_,_,_
56029,7,180,_,CD,CD,_,5,2138,180,2138,0,---,O,0,_,8,number,_,_,_
56030,8,חברי,חבר,NNT,NNT,gen=M|num=P,6,2138,חברי,2138,0,---,O,0,_,11,subj,_,_,_
56031,9,ה,ה,DEF,DEF,_,7,2138,המועצה,2138,0,---,O,0,_,10,def,_,_,_
56032,10,מועצה,מועצה,NN,NN,gen=F|num=S,7,2138,המועצה,2138,1,---,O,0,_,8,gobj,_,_,_


In [1927]:
spdf_fixed[spdf_fixed.form=='רצועה']

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped
11638,8,רצועה,רצועה,NN,NN,gen=F|num=S,4,515,ברצועה,515,2,---,E-GPE,1,GPE[22],6,pobj,_,_,GPE[22]
11749,17,רצועה,רצועה,NN,NN,gen=F|num=S,9,520,ברצועה,520,2,---,E-GPE,1,GPE[30],15,pobj,_,_,GPE[30]
12318,20,רצועה,רצועה,NN,NN,gen=F|num=S,11,547,ברצועה,547,2,---,E-GPE,1,GPE[65],18,pobj,_,_,GPE[65]
12347,21,רצועה,רצועה,NN,NN,gen=F|num=S,13,548,הרצועה,548,1,---,E-GPE,1,GPE[68],19,gobj,_,_,GPE[68]
12398,9,רצועה,רצועה,NN,NN,gen=F|num=S,6,551,ברצועה,551,2,---,E-GPE,1,GPE[69],7,pobj,_,_,GPE[69]
12690,21,רצועה,רצועה,NN,NN,gen=F|num=S,13,566,ברצועה,566,2,---,E-GPE,1,GPE[78],19,pobj,_,_,GPE[78]
12978,12,רצועה,רצועה,NN,NN,gen=F|num=S,10,579,ברצועה,579,2,---,E-GPE,1,GPE[89],10,pobj,_,_,GPE[89]
49357,47,רצועה,רצועה,NN,NN,gen=F|num=S,28,1922,רצועה,1922,0,---,O,0,_,52,conj,_,_,_
49364,54,רצועה,_,NN,NN,gen=F|num=S,31,1922,"ו""רצועה",1922,2,---,O,0,_,52,conj,_,_,_
120135,8,רצועה,רצועה,NN,NN,gen=F|num=S,6,4766,הרצועה,4766,1,---,E-LOC,1,LOC[56],6,gobj,_,_,LOC[56]


In [1931]:
spdf_fixed[spdf_fixed.global_sent_id==2138]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped
56023,1,קפידבין,_,ZVL,ZVL,_,1,2138,קפידבינפחות,2138,0,---,O,0,_,11,dep,_,_,_
56024,2,פחות,פחותJJT,JJT,gen=M|num=S,_,1,2138,קפידבינפחות,2138,1,---,O,0,_,dep,_,_,_,_
56025,3,מ,מ,PREPOSITION,PREPOSITION,_,2,2138,ממחצית,2138,0,---,O,0,_,2,dep,_,_,_
56026,4,מחצית,_,DTT,DTT,_,2,2138,ממחצית,2138,1,---,O,0,_,8,dep,_,_,_
56027,5,מתוך,מתוך,IN,IN,_,3,2138,מתוך,2138,0,---,O,0,_,8,dep,_,_,_
56028,6,כלל,כלל,DTT,DTT,_,4,2138,כלל,2138,0,---,O,0,_,8,det,_,_,_
56029,7,180,_,CD,CD,_,5,2138,180,2138,0,---,O,0,_,8,number,_,_,_
56030,8,חברי,חבר,NNT,NNT,gen=M|num=P,6,2138,חברי,2138,0,---,O,0,_,11,subj,_,_,_
56031,9,ה,ה,DEF,DEF,_,7,2138,המועצה,2138,0,---,O,0,_,10,def,_,_,_
56032,10,מועצה,מועצה,NN,NN,gen=F|num=S,7,2138,המועצה,2138,1,---,O,0,_,8,gobj,_,_,_


In [1933]:
spdf_fixed.loc[(spdf_fixed.global_sent_id==2138) & (spdf_fixed.id==2), 'deprel'] = 'dep'
spdf_fixed.loc[(spdf_fixed.global_sent_id==2138) & (spdf_fixed.id==2), 'head'] = '11'

In [1934]:
spdf_fixed[spdf_fixed.global_sent_id==5333]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped
135626,1,קפידבין,_,ZVL,ZVL,_,1,5333,קפידבינברצועה,5333,0,---,O,0,_,5,dep,_,_,_
135627,2,ב,_,PREPOSITION,PREPOSITION,_,1,5333,קפידבינברצועה,5333,1,---,O,0,_,5,prepmod,_,_,_
135628,3,ה,_,DEF,DEF,_,1,5333,קפידבינברצועה,5333,2,---,B-LOC,1,LOC[49],4,def,_,_,LOC[49]
135629,4,רצועה,רצועה,NN,NN gen=F|num=S,_,1,5333,קפידבינברצועה,5333,3,---,E-LOC,1,LOC[49],pobj,_,_,_,LOC[49]
135630,5,היתה,היה,COP,COP,gen=F|num=S|per=3,2,5333,היתה,5333,0,---,O,0,_,0,ROOT,_,_,_
135631,6,ב,ב,PREPOSITION,PREPOSITION,_,3,5333,ביום,5333,0,---,O,0,_,5,prepmod,_,_,_
135632,7,יום,יום,NNT,NNT,gen=M|num=S,3,5333,ביום,5333,1,---,O,0,_,6,pobj,_,_,_
135633,8,ששי,ששי,NNP,NNP,_,4,5333,ששי,5333,0,---,O,0,_,7,gobj,_,_,_
135634,9,שביתה,שביתה,NN,NN,gen=F|num=S,5,5333,שביתה,5333,0,---,O,0,_,5,subj,_,_,_
135635,10,כללית,כללי,JJ,JJ,gen=F|num=S,6,5333,כללית,5333,0,---,O,0,_,9,amod,_,_,_


In [1935]:
spdf_fixed.loc[(spdf_fixed.global_sent_id==5333) & (spdf_fixed.id==4), 'deprel'] = 'pobj'
spdf_fixed.loc[(spdf_fixed.global_sent_id==5333) & (spdf_fixed.id==4), 'head'] = '2'

In [1736]:
spdf_fixed.loc[spdf_fixed.feats.isin(['8','2']), 'feats'] = '_'

In [1708]:
spdf_fixed['ner_escaped'] = spdf_fixed.ner.str.replace('|', '~')

In [1912]:
spdf_fixed['misc'] = spdf_fixed.misc.fillna('_')

In [16]:
spdf_fixed = pd.read_csv('align/spdf_fixed.csv.gz', compression='gzip')

In [20]:
from collections import OrderedDict

In [27]:
DEFAULT_FIELDS = ['id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc']
def sentence_to_conllu_token_list(sent, add_misc=['biose', 'ner_escaped', 'token_id', 'token_str']):
    metadata = OrderedDict([('sent_id', str(sent['sent_id'].iat[0])), ('global_sent_id', str(sent['global_sent_id'].iat[0]))])
    
    if add_misc is not None:
        amisc = sent.apply(lambda x: [m+'='+str(x[m]) for m in add_misc], axis=1)
        amisc = amisc.str.join('|')
    token_morphs = (sent.groupby('token_id').id.agg([min, max])
                 .pipe(lambda x: x[x['max'] - x['min']>0])
                 .assign(token_morphs=lambda x: x['min'].astype(str)+'-'+x['max'].astype(str))
                 .set_index('min')['token_morphs'])   
    token_list = []
    
    for (i, row), token_str, am in zip(sent[DEFAULT_FIELDS].iterrows(), sent['token_str'].tolist(), amisc.tolist()):
        token = OrderedDict(row)
        if type(token['feats']) == str and token['feats']!='_':
            #token['feats'] = OrderedDict([f.split('=') for f in token['feats'].split('|')])
            feats = OrderedDict()
            for f in token['feats'].split('|'):
                k,v = f.split('=')
                if k not in feats:
                    feats[k] = v
                else:
                    feats[k] = feats[k]+','+v
            token['feats'] = feats
                        
        am = OrderedDict([f.split('=') for f in am.split('|') ])
        if token['misc'] =='_' or token['misc'] is None:
            token['misc'] = am
        elif type(token['misc'])==str:
            token['misc'] = OrderedDict([('MISC', token['misc'])])
            token['misc'].update(am)
        elif type(token['misc'])==OrderedDict:
            am.update(token['misc'])
            token['misc'] = am
            
        if token['id'] in token_morphs:
            tok_row = OrderedDict([(f, '_') for f in DEFAULT_FIELDS])
            id_tup = token_morphs[token['id']].split('-')
            tok_row['id'] = (id_tup[0], '-', id_tup[1])
            tok_row['form'] = token_str
            token_list.append(tok_row)
            
        token_list.append(token)
        
    return token_list, metadata



In [28]:
from conllu.models import TokenList
sp_conllu_sents = [TokenList(*sentence_to_conllu_token_list(spdf_fixed[spdf_fixed.sent_id==i])) for i in spdf_fixed.sent_id.drop_duplicates().tolist()]
print(sp_conllu_sents[2].serialize())

# sent_id = 3
# global_sent_id = 3
1	יו"ר	יו"ר	NNT	NNT	gen=M|num=S	12	subj	_	biose=O|ner_escaped=_|token_id=1|token_str=יו"ר
2-3	הוועדה	_	_	_	_	_	_	_	_
2	ה	ה	DEF	DEF	_	3	def	_	biose=O|ner_escaped=_|token_id=2|token_str=הוועדה
3	וועדה	ועדה	NN	NN	gen=F|num=S	1	gobj	_	biose=O|ner_escaped=_|token_id=2|token_str=הוועדה
4	,	_	yyCM	yyCM	_	1	punct	_	biose=O|ner_escaped=_|token_id=3|token_str=,
5	ח"כ	ח"כ	NN	NN	gen=M|num=S	1	appos	_	biose=O|ner_escaped=_|token_id=4|token_str=ח"כ
6	אורה	אורה	NNP	NNP	_	5	nn	_	biose=B-PER|ner_escaped=PER[3]|token_id=5|token_str=אורה
7	נמיר	נמיר	NNP	NNP	_	6	nn	_	biose=E-PER|ner_escaped=PER[3]|token_id=6|token_str=נמיר
8	(	_	yyLRB	yyLRB	_	5	punct	_	biose=O|ner_escaped=_|token_id=7|token_str=(
9	מערך	מערך	NNP	NNP	_	5	appos	_	biose=S-ORG|ner_escaped=ORG|token_id=8|token_str=מערך
10	)	_	yyRRB	yyRRB	_	5	punct	_	biose=O|ner_escaped=_|token_id=9|token_str=)
11	,	_	yyCM	yyCM	_	1	punct	_	biose=O|ner_escaped=_|token_id=10|token_str=,
12	טענה	טען	VB	VB	gen=F|num=S|per=3|tense=

In [29]:
for s in sp_conllu_sents:
    for t in s:
        if type(t['feats'])==OrderedDict:
            for k, v in t['feats'].items():
                if ',' in v:
                    print (k, v)

suf_gen F,M
gen F,M
gen F,M
num P,S
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
suf_gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
num P,S
gen F,M
gen F,M
num P,S
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
num D,P
gen F,M
gen F,M


In [1830]:
with open('align/spmrl_fixed.conllu', 'w', encoding='utf8') as f:
    for s in sp_conllu_sents:
        f.write(s.serialize())

In [1831]:
ud_conllu_sents = [TokenList(*sentence_to_conllu_token_list(uddf_fixed[uddf_fixed.sent_id==i])) for i in uddf_fixed.sent_id.drop_duplicates().tolist()]
print(ud_conllu_sents[2].serialize())

# sent_id = 3
# global_sent_id = 3
1	יו"ר	יו"ר	NOUN	NOUN	Abbr=Yes|Definite=Cons|Gender=Masc|Number=Sing	12	nsubj	_	biose=O|ner_escaped=_|token_id=1|token_str=יו"ר
2-3	הוועדה	_	_	_	_	_	_	_	_
2	ה	ה	DET	DET	PronType=Art	3	det:def	_	biose=O|ner_escaped=_|token_id=2|token_str=הוועדה
3	וועדה	ועדה	NOUN	NOUN	Gender=Fem|Number=Sing	1	compound:smixut	_	biose=O|ner_escaped=_|token_id=2|token_str=הוועדה
4	,	,	PUNCT	PUNCT	_	1	punct	_	biose=O|ner_escaped=_|token_id=3|token_str=,
5	ח"כ	ח"כ	NOUN	NOUN	Abbr=Yes|Gender=Masc|Number=Sing	1	appos	_	biose=O|ner_escaped=_|token_id=4|token_str=ח"כ
6	אורה	אורה	PROPN	PROPN	_	5	flat:name	_	biose=B-PER|ner_escaped=PER[3]|token_id=5|token_str=אורה
7	נמיר	נמיר	PROPN	PROPN	_	5	flat:name	_	biose=E-PER|ner_escaped=PER[3]|token_id=6|token_str=נמיר
8	(	(	PUNCT	PUNCT	_	9	punct	_	biose=O|ner_escaped=_|token_id=7|token_str=(|SpaceAfter=No
9	מערך	מערך	PROPN	PROPN	_	5	appos	_	biose=S-ORG|ner_escaped=ORG|token_id=8|token_str=מערך|SpaceAfter=No
10	)	)	PUNCT	PUNCT	_	9	punct	_	bi

In [1832]:
with open('align/ud_fixed.conllu', 'w', encoding='utf8') as f:
    for s in ud_conllu_sents:
        f.write(s.serialize())

## Alignment - Add text metadata (tokenized SPMRL, pre-tokenized UD)

In [32]:
align_map = sentence_align_map

In [34]:
sentence_align_map.head()

,ud,spmrl,shft,shift_change,global_sent_id,spmrl_very_similar,ud_very_similar,spmrl_duplicates,ud_duplicates
0,1,1,0,False,1,NaN,NaN,NaN,NaN
1,2,2,0,False,2,NaN,NaN,NaN,NaN
2,3,3,0,False,3,NaN,NaN,NaN,NaN
3,4,4,0,False,4,NaN,NaN,NaN,NaN
4,5,5,0,False,5,NaN,NaN,NaN,NaN


In [38]:
ud_tl = ud_dev + ud_train + ud_test
ud_text = {tl.metadata['sent_id']: tl.metadata['text'] for tl in ud_tl}
global_ud_text = {align_map.loc[align_map.ud.astype(str)==k, 'global_sent_id'].values[0]: v for k,v in ud_text.items() if len(align_map.loc[align_map.ud.astype(str)==k, 'global_sent_id'])!=0 }
global_ud_text

{1: 'עשרות אנשים מגיעים מתאילנד לישראל כשהם נרשמים כמתנדבים, אך למעשה משמשים עובדים שכירים זולים.',
 2: 'תופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת, שדנה בנושא העסקת עובדים זרים.',
 3: 'יו"ר הוועדה, ח"כ אורה נמיר (מערך), טענה כי "מביאים עובדים זרים לישראל על תקן של מתנדבים מתאילנד, רק כדי לא לשלם להם שכר מינימום.',
 4: 'מצד אחד רוצה האוצר להוריד את שכר המינימום, ומצד שני מתיר משרד העבודה והרווחה להעסיק עובדים זרים בפחות משכר זה".',
 5: 'נמיר הודיעה כי תפנה לשרי הפנים והעבודה והרווחה ולמזכיר תנועת המושבים, בתביעה לבטל את הזמנתם של 500 עובדים זרים מתאילנד כמתנדבים כביכול.',
 6: 'היא הודיעה כי הוועדה תגבש הצעת חוק בנושא העובדים הזרים, שתכלול איסור על מתן שכר לעובדים מתחת לשכר המינימום ומתן התנאים הסוציאליים המקובלים במקום העבודה.',
 7: 'כמו כן, תציב הצעת החוק עונשי מאסר והטלת קנסות כבדים למי שיעסיק עובדים זרים בלא רשיון.',
 8: 'מרגלית אילת, הממונה על מתן היתרי עבודה לזרים בשירות התעסוקה, מסרה כי תנועת המושבים הפעילה לחץ שיותר לה להביא עובדים זרים מתאילנד.',
 9: 'היא אמרה כי שירו

In [39]:
for tl in sp_conllu_sents:
    tl.metadata['text_from_ud'] = global_ud_text[int(tl.metadata['global_sent_id'])]

In [1845]:
for tl in ud_conllu_sents:
    tl.metadata['text'] = global_ud_text[int(tl.metadata['global_sent_id'])]

In [15]:
with open('align/spmrl_fixed.conllu', 'w', encoding='utf8') as f:
    for s in sp_conllu_sents:
        f.write(s.serialize())

In [1847]:
with open('align/ud_fixed.conllu', 'w', encoding='utf8') as f:
    for s in ud_conllu_sents:
        f.write(s.serialize())

## Alignment - Add duplicate metadata

In [1848]:
sentence_align_map.head()

,ud,spmrl,shft,shift_change,global_sent_id,spmrl_very_similar,ud_very_similar,spmrl_duplicates,ud_duplicates
0,1,1,0,False,1,NaN,NaN,NaN,NaN
1,2,2,0,False,2,NaN,NaN,NaN,NaN
2,3,3,0,False,3,NaN,NaN,NaN,NaN
3,4,4,0,False,4,NaN,NaN,NaN,NaN
4,5,5,0,False,5,NaN,NaN,NaN,NaN


In [40]:
for tl in sp_conllu_sents:
    tl.metadata['very_similar_sent_id'] = str (sentence_align_map.loc[sentence_align_map.spmrl==int(tl.metadata['sent_id']), 'spmrl_very_similar'].fillna('{}').values[0] )
    tl.metadata['duplicate_sent_id'] = str (sentence_align_map.loc[sentence_align_map.spmrl==int(tl.metadata['sent_id']), 'spmrl_duplicates'].fillna('{}').values[0] )

In [1872]:
for tl in ud_conllu_sents:
    vs = sentence_align_map.loc[sentence_align_map.ud==tl.metadata['sent_id'], 'ud_very_similar'].fillna('{}').values[0]
    if vs!='{}':
        vs = str(set([int(x) for x in vs]))
        print (vs)
        
    tl.metadata['very_similar_sent_id'] = vs
    dup = sentence_align_map.loc[sentence_align_map.ud==tl.metadata['sent_id'], 'ud_duplicates'].fillna('{}').values[0]
    if dup!='{}':
        dup = str(set([int(x) for x in dup]))
        print (dup)
    tl.metadata['duplicate_sent_id'] = dup

{326}
{296}
{554}
{555}
{530}
{531}
{2837}
{861}
{819}
{813}
{617}
{4058}
{5531}
{1616}
{5164}
{5817}
{2315}
{2316}
{2317}
{2318}
{5282, 2405, 3785, 5002, 4941, 2288, 5169, 3379, 2325, 5020, 3549}
{5169, 5282, 3379}
{3749, 5099, 5071, 3314, 4210}
{2235}
{2234}
{5282, 2405, 3785, 5002, 4941, 5169, 3379, 2325, 2106, 5020, 3549}
{3785, 2405, 3549}
{1913}
{1914}
{1915}
{1916}
{5282, 2405, 3785, 5002, 4941, 2288, 5169, 3379, 2106, 5020, 3549}
{5379}
{2753, 2555, 2749}
{2555, 2749}
{2790, 2807}
{2753, 2749, 2526}
{2555, 2749}
{2753, 2555, 2526}
{2555, 2749, 2526}
{2786, 2804}
{2754, 2804}
{2529, 2807}
{2815}
{2754, 2786}
{2529, 2790}
{2799}
{3082}
{3078}
{3128}
{3127}
{3301, 4069}
{3619}
{3616}
{5564}
{5432}
{5432}
{5433}
{5434}
{5435}
{5436}
{5436}
{5437}
{5439}
{5440}
{5441}
{5443}
{5444}
{5444}
{5445}
{5446}
{5447}
{5447}
{5448}
{5449}
{5449}
{5450}
{5450}
{5451}
{5451}
{5452}
{5452}
{5454}
{5455}
{5456}
{5457}
{5458}
{5459}
{5461}
{5461}
{5462}
{5462}
{5463}
{5467}
{5468}
{5468}
{5469}
{

## Get back MISC from UD token lines (SpaceAfter)

In [1879]:
ud_token_only_misc = {}
for tl in ud_tl:
    for t in tl:
        if type(t['id'])==tuple and t['misc'] is not None:
            ud_token_only_misc[(tl.metadata['sent_id'], t['id'])] = t['misc']

In [ ]:
for tl in ud_conllu_sents:
    for t in tl:
        if type(t['id'])==tuple and (tl.metadata['sent_id'], (int(t['id'][0]), '-', int(t['id'][2]))) in ud_token_only_misc:
            t['misc'] = ud_token_only_misc[(tl.metadata['sent_id'], (int(t['id'][0]), '-', int(t['id'][2])))]
            

In [1886]:
with open('align/ud_fixed.conllu', 'w', encoding='utf8') as f:
    for s in ud_conllu_sents:
        f.write(s.serialize())

In [1894]:
with open('align/ud_fixed.conllu', 'r', encoding='utf8') as f:
    ud_conllu_check = parse(f.read())

In [1942]:
with open('align/spmrl_fixed.conllu', 'r', encoding='utf8') as f:
    sp_conllu_check = parse(f.read())

## Add token metadata

In [1945]:
sp_token_join = spdf_fixed.groupby('global_sent_id').apply(lambda x: ' '.join(x.drop_duplicates(subset='token_id').token_str.tolist()))
ud_token_join = uddf_fixed.groupby('global_sent_id').apply(lambda x: ' '.join(x.drop_duplicates(subset='token_id').token_str.tolist()))

In [1949]:
sp_token_join[1]

'עשרות אנשים מגיעים מתאילנד לישראל כשהם נרשמים כמתנדבים , אך למעשה משמשים עובדים שכירים זולים .'

## Add dev-train-test metadata

In [47]:
sp_dev.orig_sent.min(), sp_dev.orig_sent.max(), sp_train.orig_sent.min(), sp_train.orig_sent.max(), sp_test.orig_sent.min(), sp_test.orig_sent.max()


(1, 500, 501, 5437, 5438, 6153)

In [48]:
spdf_fixed[spdf_fixed.global_sent_id!=spdf_fixed.sent_id]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped


In [6]:
def get_set_from_global_sent_id(gsi):
    if gsi>=1 and gsi<=500:
        return 'dev'
    elif gsi>=501 and gsi<=5437:
        return 'train'
    elif gsi>=5438 and gsi<=6153:
        return 'test'
    

In [50]:
for tl in sp_conllu_sents:
    tl.metadata['set'] = get_set_from_global_sent_id(int(tl.metadata['global_sent_id']))

In [1963]:
for tl in ud_conllu_sents:
    tl.metadata['set'] = get_set_from_global_sent_id(int(tl.metadata['global_sent_id']))

In [54]:
with open('align/spmrl_fixed.conllu', 'w', encoding='utf8') as f:
    for s in sp_conllu_sents:
        f.write(s.serialize())

In [1965]:
with open('align/ud_fixed.conllu', 'w', encoding='utf8') as f:
    for s in ud_conllu_sents:
        f.write(s.serialize())

In [53]:
sp_conllu_sents[0].metadata

OrderedDict([('sent_id', '1'),
             ('global_sent_id', '1'),
             ('text_from_ud',
              'עשרות אנשים מגיעים מתאילנד לישראל כשהם נרשמים כמתנדבים, אך למעשה משמשים עובדים שכירים זולים.'),
             ('very_similar_sent_id', '{}'),
             ('duplicate_sent_id', '{}'),
             ('set', 'dev')])

In [3]:
import pandas as pd

In [9]:
spdf_fixed = pd.read_csv('align/spdf_fixed.csv.gz', compression='gzip')

In [10]:
spdf_fixed['set'] = spdf_fixed.global_sent_id.map(get_set_from_global_sent_id)

In [11]:
spdf_fixed.to_csv('align/spdf_fixed.csv.gz', compression='gzip', index=False)

In [12]:
spdf_fixed.shape

(155330, 21)

## 4. Create tests to run when commiting new versions of TBs 

## 5. Upload official train-dev-test splits for SPMRL & UD
   1. Standard sentence indexes 
   1. Standard token indexes (when possible) 
   1. CONLL
   1. NER

In [1891]:
spdf_fixed[spdf_fixed.form=='חוויותיו']

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped
26725,28,חוויותיו,חוויה,NN,NN_S_PP,gen=F|num=P|suf_gen=M|suf_num=S|suf_per=3,22,1096,חוויותיו,1096,0,---,O,0,_,27,pobj,_,_,_
74598,38,חוויותיו,חוויה,NN,NN_S_PP,gen=F|num=P|suf_gen=M|suf_num=S|suf_per=3,32,2945,חוויותיו,2945,0,---,O,0,_,37,obj,_,_,_


In [1892]:
spdf_fixed[spdf_fixed.global_sent_id==1096]

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,token_morph_id,binyan,biose,ner_layers,ner,head,deprel,deps,misc,ner_escaped
26698,1,ה,ה,DEF,DEF,_,1,1096,הסיפור,1096,0,---,O,0,_,2,def,_,_,_
26699,2,סיפור,סיפור,NN,NN,gen=M|num=S,1,1096,הסיפור,1096,1,---,O,0,_,26,subj,_,_,_
26700,3,",",_,yyCM,yyCM,_,2,1096,",",1096,0,---,O,0,_,2,punct,_,_,_
26701,4,כמו,כמו,IN,IN,_,3,1096,כמו,1096,0,---,O,0,_,2,prepmod,_,_,_
26702,5,שלושה,שלושה,CD,CD,gen=M|num=S,4,1096,שלושה,1096,0,---,O,0,_,4,number,_,_,_
26703,6,אחרים,אחר,JJ,JJ,gen=M|num=P,5,1096,אחרים,1096,0,---,O,0,_,4,amod,_,_,_
26704,7,"""",_,yyQUOT,yyQUOT,_,6,1096,"""",1096,0,---,O,0,_,4,punct,_,_,_
26705,8,ב,ב,PREPOSITION,PREPOSITION,_,7,1096,ביום,1096,0,---,B-WOA,1,WOA[56],4,hd,_,_,WOA[56]
26706,9,יום,יום,NNT,NNT,gen=M|num=S,7,1096,ביום,1096,1,---,I-WOA,1,WOA[56],8,pobj,_,_,WOA[56]
26707,10,מותו,מוות,NN,NN_S_PP,gen=M|num=S|suf_gen=M|suf_num=S|suf_per=3,8,1096,מותו,1096,0,---,E-WOA,1,WOA[56],9,gobj,_,_,WOA[56]
